In [17]:
import pymysql
import requests
import re
import pandas as pd 
import numpy as np 
import random
from pyecharts import options as opts
from pyecharts.charts import Map 
from pyecharts.charts import Line
from pyecharts.charts import Polar
from pyecharts.charts import Bar
from pyecharts.charts import Page


# 连接数据库
conn = pymysql.connect(host='localhost',user='root',password='123456',
                       database='activity_analysic',port=3306,charset='gbk')
#创建游标
cur = conn.cursor()

# 柱状图

## 车辆出行平均车速分布/监测放电完整过程 SOC 变化状态/充电状态车辆总数

展示每一天车辆出行平均车速的车辆统计分布

每一天车辆的 SOC 平均取值

可充电储能温度探针个数每一天占比

展示每一天各充电状态的车辆总数

In [18]:
# 出行平均车速分布
sql1 = '''
select 
	date_format(sendtime, "%Y-%m-%d") as time,
	avg(speed) avg_speed
from 
	activity a 
group by
	date_format(sendtime, "%Y-%m-%d")
order by 
	time ;
'''
cur.execute(sql1)
result1 = cur.fetchall()
result1 = pd.DataFrame(list(result1),columns=['time','平均车速'])

# SOC 变化状态
sql2 = '''
select 
	date_format(sendtime, "%Y-%m-%d") as time,
	avg(SOC) avg_SOC
from 
	activity a 
group by
	date_format(sendtime, "%Y-%m-%d")
order by 
	time ;
'''
cur.execute(sql2)
result2 = cur.fetchall()
result2 = pd.DataFrame(list(result2),columns=['time','平均SOC'])

sql3 = '''
select
	date_format(sendtime, "%Y-%m-%d") as time,
	count(total_TEMP_probe) as '可充电储能温度探针个数均值'
from 
	activity a
group by
	date_format(sendtime, "%Y-%m-%d")
order by 
	time;	
'''
cur.execute(sql3)
result3 = cur.fetchall()
result3 = pd.DataFrame(list(result3),columns=['time','可充电储能温度探针个数均值'])
result3['可充电储能温度探针个数均值'] = result3['可充电储能温度探针个数均值']/100

# 出行平均车速分布
sql4 = '''
with charge_state_tb as(
    select 
    	date_format(sendtime, "%Y-%m-%d") as time,
    	charge_state,
		count(distinct vin) as num
    from 
    	activity 
    group by
		date_format(sendtime, "%Y-%m-%d"),charge_state
	order by 
		time
)
select 
	a.time,
	a.num '停车充电',
	b.num  '行驶充电',
	c.num '未充电状态',
	d.num '充电完成'
from 
	(
		select 
			time,num 
		from 
			charge_state_tb 
		where
			charge_state = 1
	) a 
join 
	(
		select 
			time,num
		from 
			charge_state_tb
		where
			charge_state = 2
	) b
on 
	a.time = b.time
join 
	(
		select 
			time,num
		from 
			charge_state_tb
		where
			charge_state = 3
	) c
on 
	a.time = c.time
join 
	(
		select 
			time,num
		from 
			charge_state_tb
		where
			charge_state = 4
	) d
on 
	a.time = d.time
'''
cur.execute(sql4)
result4 = cur.fetchall()
result4 = pd.DataFrame(list(result4),columns=['time','停车充电','行驶充电','未充电状态','充电完成'])
result4['行驶充电'] = result4['行驶充电'] * 50
result4['充电完成'] = result4['充电完成'] * 10

# 合并三张表
result = pd.merge(result1,result2,on='time')
result = pd.merge(result,result3,on='time')
result = pd.merge(result,result4,on='time')

# 柱状图创建
bars = Bar(init_opts=opts.InitOpts(theme='dark')) 

# 添加值
bars.add_xaxis(list(result['time'])),
bars.add_yaxis("平均车速", list(result['平均车速'])),
bars.add_yaxis("平均SOC", list(result['平均SOC'])),
bars.add_yaxis("可充电储能温度探针个数均值", list(result['可充电储能温度探针个数均值'])),
bars.add_yaxis("停车充电", list(result['停车充电'])),
bars.add_yaxis("行驶充电", list(result['行驶充电'])),
bars.add_yaxis("未充电状态", list(result['未充电状态'])),
bars.add_yaxis("充电完成", list(result['充电完成'])),

# 系列配置
bars.set_series_opts(
    
    # 标签
    label_opts=opts.LabelOpts(
        # 是否显示标签
        is_show=False
    ),
)

# 全局配置
bars.set_global_opts(
        
    # 图表标题
    title_opts=opts.TitleOpts(
        # 标题
        title="车速/SOC/探针数/充电状态",
        # 标题位置
        pos_left="left",
        # 标题字体大小设置为 20
        title_textstyle_opts=opts.TextStyleOpts(font_size=15)
    ),
        
    # 图例设置
    legend_opts=opts.LegendOpts(
        # 显示图例，就是各个指标的切换按钮
        is_show=True, 
        # 图例设置为单选模式(multiple 多选)
        selected_mode='multiple',
        # 图例的位置
        pos_top='5%', pos_right='10%',
        # 图例的布局朝向，水平(vertical 垂直)
        orient='horizontal', 
        # 图例形状
        legend_icon='circle'
    ),
        
    # 工具箱设置
    toolbox_opts=opts.ToolboxOpts(
        # 是否显示工具栏组件
        is_show=True,
        # 布局朝向('vertical')
        orient='vertical',
        # 离上边距的距离
        pos_top='10%',
        # 离左边距的距离
        pos_left='91%',
            
        # 配置各个工具箱
        feature=opts.ToolBoxFeatureOpts(
                
            # 保存工具
            opts.ToolBoxFeatureSaveAsImageOpts(
                # 是否显示
                is_show=True,
                # 提示语
                title="保存",                 
            ),
                
            # 还原工具
            opts.ToolBoxFeatureRestoreOpts(
                # 是否显示该工具
                is_show=True,
                # 提示语
                title="还原", 
            ),
                
            # 数据视图工具
            opts.ToolBoxFeatureDataViewOpts(
                # 是否显示该工具
                is_show=True,
                # 提示语
                title="数据视图",
                # 是否不可编辑
                is_read_only=False,
            ),
            
            # 缩放工具配置项，直角坐标系适用
            opts.ToolBoxFeatureDataZoomOpts(
                # 是否显示该工具。
                is_show=True,
                # 提示语
                zoom_title="区域缩放",
                # 提示语
                back_title="缩放还原",
            ),
                
            # 图表类型切换，适用于直角坐标系
            opts.ToolBoxFeatureMagicTypeOpts(
                # 是否显示该工具
                is_show=True,
                # 启用的动态类型('stack','line','bar','tiled')
                type_=['stack','line','bar','tiled'],
                # 折线标题文本
                line_title="折线图",
                # 柱状标题文本
                bar_title="柱状图",
                # 堆积标题文本
                stack_title="堆叠",
                # 平铺标题文
                tiled_title="平铺",
            ),
                
            # 工具箱选框组件配置项
            opts.ToolBoxFeatureBrushOpts(  
                # 选择显示哪些选框
                type_=[]
            ),
        )
    ),
)
bars.render_notebook()

D:\programming\dev_env\anaconda\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


# 堆积柱状图

## 运营车辆状态分布占比分析/运行模式车辆分布统计/驱动电机状态分布分析

展示每一天的车辆状态为”车辆启动状态”的车辆数

统计每一天运行模式

展示每一天各种驱动机状态总数

In [19]:
# 车辆状态
sql1 = '''
with vehicle_status_tb as(
    select 
    	date_format(sendtime, "%Y-%m-%d") as time,
    	vehicle_status,
		count(distinct vin) as num
    from 
    	activity 
    group by
		date_format(sendtime, "%Y-%m-%d"),vehicle_status
	order by 
		time
)
select 
	a.time,
	a.num '车辆启动状态',
	b.num  '熄火',
	c.num '其他状态'
from 
	(
		select 
			time,num 
		from 
			vehicle_status_tb 
		where
			vehicle_status = 1
	) a 
join 
	(
		select 
			time,num
		from 
			vehicle_status_tb 
		where
			vehicle_status = 2
	) b
on 
	a.time = b.time
join 
	(
		select 
			time,num
		from 
			vehicle_status_tb 
		where
			vehicle_status = 3
	) c
on 
	a.time = c.time
'''
cur.execute(sql1)
result1 = cur.fetchall()
result1 = pd.DataFrame(list(result1),columns=['time','启动状态','熄火状态','其他状态'])

# 车辆模式
sql2 = '''
with run_model_tb as(
    select 
    	date_format(sendtime, "%Y-%m-%d") as time,
    	run_model,
		count(distinct vin) as num
    from 
    	activity 
    group by
		date_format(sendtime, "%Y-%m-%d"),run_model
	order by 
		time
)
select 
	a.time,
	a.num '纯点',
	b.num  '混动'
from 
	(
		select 
			time,num 
		from 
			run_model_tb
		where
			run_model = 1
	) a 
join 
	(
		select 
			time,num
		from 
			run_model_tb
		where
			run_model = 2
	) b
on 
	a.time = b.time
'''
cur.execute(sql2)
result2 = cur.fetchall()
result2 = pd.DataFrame(list(result2),columns=['time','纯点','混动'])

# 驱动机
sql3 = '''
with dm_state_tb as(
    select 
    	date_format(sendtime, "%Y-%m-%d") as time,
    	DM_status,
		count(distinct vin) as num
    from 
    	activity 
    group by
		date_format(sendtime, "%Y-%m-%d"),DM_status
	order by 
		time
)
select 
	a.time,
	a.num '耗电',
	b.num  '发电',
	c.num '关闭状态',
	d.num '准备状态'
from 
	(
		select 
			time,num 
		from 
			dm_state_tb 
		where
			DM_status = 1
	) a 
join 
	(
		select 
			time,num
		from 
			dm_state_tb 
		where
			DM_status = 2
	) b
on 
	a.time = b.time
join 
	(
		select 
			time,num
		from 
			dm_state_tb 
		where
			DM_status = 3
	) c
on 
	a.time = c.time
join 
	(
		select 
			time,num
		from 
			dm_state_tb 
		where
			DM_status = 4
	) d
on 
	a.time = d.time
'''
cur.execute(sql3)
result3 = cur.fetchall()
result3 = pd.DataFrame(list(result3),columns=['time','耗电','发电','关闭状态',
                                            '准备状态'])

# 合并表
result = pd.merge(result1,result2,on='time')
result = pd.merge(result,result3,on='time')

# 柱状图创建
bars = Bar(init_opts=opts.InitOpts(theme='dark')) 

# 添加值
bars.add_xaxis(list(result['time'])),
bars.add_yaxis("启动状态", list(result['启动状态']),stack="stack1"),
bars.add_yaxis("熄火状态", list(result['熄火状态']),stack="stack1"),
bars.add_yaxis("其他状态", list(result['其他状态']),stack="stack1"),
bars.add_yaxis("纯点模式", list(result['纯点']),stack="stack1"),
bars.add_yaxis("混动模式", list(result['混动']),stack="stack1"),
bars.add_yaxis("耗电状态", list(result['耗电']),stack="stack1"),
bars.add_yaxis("发电状态", list(result['发电']),stack="stack1"),
bars.add_yaxis("关闭状态", list(result['关闭状态']),stack="stack1"),
bars.add_yaxis("准备状态", list(result['准备状态']),stack="stack1"),
    
# 配置
bars.set_global_opts(
        
    # 图表标题
    title_opts=opts.TitleOpts(
        # 标题
        title="车辆状态/模式",
        # 标题位置
        pos_left="left",
        # 标题字体大小设置为 20
        title_textstyle_opts=opts.TextStyleOpts(font_size=20)
    ),
        
    # 图例设置
    legend_opts=opts.LegendOpts(
        # 显示图例，就是各个指标的切换按钮
        is_show=True, 
        # 图例设置为单选模式(multiple 多选)
        selected_mode='multiple',
        # 图例的位置
        pos_top='5%', pos_right='10%',
        # 图例的布局朝向，水平(vertical 垂直)
        orient='horizontal', 
        # 图例形状
        legend_icon='circle'
    ),
        
    # 工具箱设置
    toolbox_opts=opts.ToolboxOpts(
        # 是否显示工具栏组件
        is_show=True,
        # 布局朝向('vertical')
        orient='vertical',
        # 离上边距的距离
        pos_top='10%',
        # 离左边距的距离
        pos_left='91%',
            
        # 配置各个工具箱
        feature=opts.ToolBoxFeatureOpts(
                
            # 保存工具
            opts.ToolBoxFeatureSaveAsImageOpts(
                # 是否显示
                is_show=True,
                # 提示语
                title="保存",                 
            ),
                
            # 还原工具
            opts.ToolBoxFeatureRestoreOpts(
                # 是否显示该工具
                is_show=True,
                # 提示语
                title="还原", 
            ),
                
            # 数据视图工具
            opts.ToolBoxFeatureDataViewOpts(
                # 是否显示该工具
                is_show=True,
                # 提示语
                title="数据视图",
                # 是否不可编辑
                is_read_only=False,
            ),
            
            # 缩放工具配置项，直角坐标系适用
            opts.ToolBoxFeatureDataZoomOpts(
                # 是否显示该工具。
                is_show=True,
                # 提示语
                zoom_title="区域缩放",
                # 提示语
                back_title="缩放还原",
            ),
                
            # 图表类型切换，适用于直角坐标系
            opts.ToolBoxFeatureMagicTypeOpts(
                # 是否显示该工具
                is_show=True,
                # 启用的动态类型('stack','line','bar','tiled')
                type_=['stack','line','bar','tiled'],
                # 折线标题文本
                line_title="折线图",
                # 柱状标题文本
                bar_title="柱状图",
                # 堆积标题文本
                stack_title="堆叠",
                # 平铺标题文
                tiled_title="平铺",
            ),
                
            # 工具箱选框组件配置项
            opts.ToolBoxFeatureBrushOpts(  
                # 选择显示哪些选框
                type_=[]
            ),
        )
    ),
)
bars.render_notebook()

D:\programming\dev_env\anaconda\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


# 折线图

## 每天接入车辆总数走势/车辆行驶里程分析

展示每一天收集到的接入车辆总数

展示每一天每辆车的平均累计里程

In [20]:
# 接入车辆总数走势
sql1 = '''
select
	date_format(sendtime, "%Y-%m-%d") as time,
	count(distinct vin) as num
from
	activity a
group by
	date_format(sendtime, "%Y-%m-%d")
order by
	time;
'''
cur.execute(sql1)
result1 = cur.fetchall()
result1 = pd.DataFrame(list(result1),columns=['time','接入数'])

# 车辆行驶里程
sql2 = '''
select
	date_format(sendtime, "%Y-%m-%d") as time,
	(sum(accu_mile) / count(distinct vin)) as mile
from 
	activity a
group by
	date_format(sendtime, "%Y-%m-%d")
order by 
	time;
'''
cur.execute(sql2)
result2 = cur.fetchall()
result2 = pd.DataFrame(list(result2),columns=['time','累计里程'])

# 加速踏板行程值
sql3 = '''
select
	date_format(sendtime, "%Y-%m-%d") as time,
	(sum(accel_pedal_travel) / count(distinct vin)) as '加速踏板行程均值'
from 
	activity a
group by
	date_format(sendtime, "%Y-%m-%d")
order by 
	time;	
'''
cur.execute(sql3)
result3 = cur.fetchall()
result3 = pd.DataFrame(list(result3),columns=['time','加速踏板行程均值'])

sql4 = '''
select
	date_format(sendtime, "%Y-%m-%d") as time,
	avg(max_bat_mer_V) as '电压最高均值',
	avg(min_bat_mer_V) as '电压最低均值'
from 
	activity a
group by
	date_format(sendtime, "%Y-%m-%d")
order by 
	time;		
'''
cur.execute(sql4)
result4 = cur.fetchall()
result4 = pd.DataFrame(list(result4),columns=['time','电压最高均值','电压最低均值'])
result4['电压最低均值'] = result4['电压最低均值'] - 1000 

# 合并四张表
result = pd.merge(result1,result2,on='time')
result = pd.merge(result,result3,on='time')
result = pd.merge(result,result4,on='time')

# 折线图创建
lines = Line(init_opts=opts.InitOpts(theme='dark')) 

# 添加值
lines.add_xaxis(list(result['time'])),
lines.add_yaxis("接入数", list(result['接入数'])),
lines.add_yaxis("累计里程", list(result['累计里程'])),
lines.add_yaxis("加速踏板行程均值", list(result['加速踏板行程均值'])),
lines.add_yaxis("电压最高均值", list(result['电压最高均值'])),
lines.add_yaxis("电压最低均值", list(result['电压最低均值'])),
    
# 配置
lines.set_global_opts(
        
    # 图表标题
    title_opts=opts.TitleOpts(
        # 标题
        title="接入数/累计里程/踏板行程/电压",
        # 标题位置
        pos_left="left",
        # 标题字体大小设置为 20
        title_textstyle_opts=opts.TextStyleOpts(font_size=15)
    ),
        
    # 图例设置
    legend_opts=opts.LegendOpts(
        # 显示图例，就是各个指标的切换按钮
        is_show=True, 
        # 图例设置为单选模式(multiple 多选)
        selected_mode='single',
        # 图例的位置
        pos_top='5%', pos_right='20%',
        # 图例的布局朝向，水平(vertical 垂直)
        orient='horizontal', 
        # 图例形状
        legend_icon='circle'
    ),
        
    # 工具箱设置
    toolbox_opts=opts.ToolboxOpts(
        # 是否显示工具栏组件
        is_show=True,
        # 布局朝向('vertical')
        orient='vertical',
        # 离上边距的距离
        pos_top='10%',
        # 离左边距的距离
        pos_left='91%',
            
        # 配置各个工具箱
        feature=opts.ToolBoxFeatureOpts(
                
            # 保存工具
            opts.ToolBoxFeatureSaveAsImageOpts(
                # 是否显示
                is_show=True,
                # 提示语
                title="保存",                 
            ),
                
            # 还原工具
            opts.ToolBoxFeatureRestoreOpts(
                # 是否显示该工具
                is_show=True,
                # 提示语
                title="还原", 
            ),
                
            # 数据视图工具
            opts.ToolBoxFeatureDataViewOpts(
                # 是否显示该工具
                is_show=True,
                # 提示语
                title="数据视图",
                # 是否不可编辑
                is_read_only=False,
            ),
            
            # 缩放工具配置项，直角坐标系适用
            opts.ToolBoxFeatureDataZoomOpts(
                # 是否显示该工具。
                is_show=True,
                # 提示语
                zoom_title="区域缩放",
                # 提示语
                back_title="缩放还原",
            ),
                
            # 图表类型切换，适用于直角坐标系
            opts.ToolBoxFeatureMagicTypeOpts(
                # 是否显示该工具
                is_show=True,
                # 启用的动态类型('stack','line','bar','tiled')
                type_=['stack','line','bar','tiled'],
                # 折线标题文本
                line_title="折线图",
                # 柱状标题文本
                bar_title="柱状图",
                # 堆积标题文本
                stack_title="堆叠",
                # 平铺标题文
                tiled_title="平铺",
            ),
                
            # 工具箱选框组件配置项
            opts.ToolBoxFeatureBrushOpts(  
                # 选择显示哪些选框
                type_=[]
            ),
        )
    ),
)
lines.render_notebook()

D:\programming\dev_env\anaconda\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


# 面积折线图

## 电池单体最高，最低温度/单体电池电压，电流，电阻/驱动电机分析

展示每一天单体电池平均最高，最低温度

单体电池平均，总电压，总电流，可充电储能装置电压，可充电储能装置电流，绝缘电阻

电机控制器直流母线电流每天平均电流，控制器平均温度，平均转速，平均转矩，平均温度，平均输入电压

In [21]:
sql1 = '''
select
	date_format(sendtime, "%Y-%m-%d") as time,
	avg(max_TEMP) as '最高温度均值',
	avg(min_TEMP) as '最低温度均值'
from 
	activity a
group by
	date_format(sendtime, "%Y-%m-%d")
order by 
	time;			
'''
cur.execute(sql1)
result1 = cur.fetchall()
result1 = pd.DataFrame(list(result1),columns=['time','最高温度均值','最低温度均值'])
result1['最低温度均值'] = result1['最低温度均值'] - 10

sql2 = '''
select
	date_format(sendtime, "%Y-%m-%d") as time,
	avg(total_voltage) as '总电压均值',
	avg(total_current) as '总电流均值',
	avg(voltage) as '可充电储能装置电压均值',
	avg(current) as '可充电储能装置电流均值',
	avg(IR) as '绝缘电阻均值'
from 
	activity a
group by
	date_format(sendtime, "%Y-%m-%d")
order by 
	time;					
'''
cur.execute(sql2)
result2 = cur.fetchall()
result2 = pd.DataFrame(list(result2),columns=['time','总电压均值','总电流均值','可充电储能装置电压均值','可充电储能装置电流均值','绝缘电阻均值'])

sql3 = '''
select 
	date_format(sendtime, "%Y-%m-%d") as time,
	avg(motor_CON_DC_bus_C) '平均电流',
	avg(DM_CON_TEMP) '控制器平均温度',
	avg(DM_speed) '平均转速',
	avg(DM_torque) '平均转矩',
	avg(DM_TEMP) '均温度',
	avg(motor_CON_in_V) '平均输入电压'
from 
	activity a 
group by
	date_format(sendtime, "%Y-%m-%d")
order by 
	time ;					
'''
cur.execute(sql3)
result3 = cur.fetchall()
result3 = pd.DataFrame(list(result3),columns=['time','平均电流','控制器平均温度','平均转速','平均转矩','均温度','平均输入电压'])

# 合并表
result = pd.merge(result1,result2,on='time')
result = pd.merge(result,result3,on='time')

# 折线图创建
lines = Line(init_opts=opts.InitOpts(theme='dark')) 

# 添加值
lines.add_xaxis(list(result['time'])),
lines.add_yaxis("最高温度均值", 
                list(result['最高温度均值']),
                # 面积堆叠
                areastyle_opts=opts.AreaStyleOpts(
                    opacity=0.3
                ),
               ),
lines.add_yaxis("最低温度均值", 
                list(result['最低温度均值']),
                # 面积堆叠
                areastyle_opts=opts.AreaStyleOpts(
                    opacity=0.3
                ),
               ),
lines.add_yaxis("总电压均值", 
                list(result['总电压均值']),
                # 面积堆叠
                areastyle_opts=opts.AreaStyleOpts(
                    opacity=0.3
                ),
               ),
lines.add_yaxis("总电流均值", 
                list(result['总电流均值']),
                # 面积堆叠
                areastyle_opts=opts.AreaStyleOpts(
                    opacity=0.3
                ),
               ),
lines.add_yaxis("平均输入电压", 
                list(result['平均输入电压']),
                # 面积堆叠
                areastyle_opts=opts.AreaStyleOpts(
                    opacity=0.3
                ),
               ),
lines.add_yaxis("可充电储能装置电压均值", 
                list(result['可充电储能装置电压均值']),
                # 面积堆叠
                areastyle_opts=opts.AreaStyleOpts(
                    opacity=0.3
                ),
               ),
lines.add_yaxis("可充电储能装置电流均值", 
                list(result['可充电储能装置电流均值']),
                # 面积堆叠
                areastyle_opts=opts.AreaStyleOpts(
                    opacity=0.3
                ),
               ),
lines.add_yaxis("绝缘电阻均值", 
                list(result['绝缘电阻均值']),
                # 面积堆叠
                areastyle_opts=opts.AreaStyleOpts(
                    opacity=0.3
                ),
               ),
lines.add_yaxis("平均电流", 
                list(result['平均电流']),
                # 面积堆叠
                areastyle_opts=opts.AreaStyleOpts(
                    opacity=0.3
                ),
               ),
lines.add_yaxis("控制器平均温度", 
                list(result['控制器平均温度']),
                # 面积堆叠
                areastyle_opts=opts.AreaStyleOpts(
                    opacity=0.3
                ),
               ),
lines.add_yaxis("平均转速", 
                list(result['平均转速']),
                # 面积堆叠
                areastyle_opts=opts.AreaStyleOpts(
                    opacity=0.3
                ),
               ),
lines.add_yaxis("平均转矩", 
                list(result['平均转矩']),
                # 面积堆叠
                areastyle_opts=opts.AreaStyleOpts(
                    opacity=0.3
                ),
               ),
lines.add_yaxis("平均温度", 
                list(result['均温度']),
                # 面积堆叠
                areastyle_opts=opts.AreaStyleOpts(
                    opacity=0.3
                ),
               ),

# 系列配置
lines.set_series_opts(
    
    # 标签
    label_opts=opts.LabelOpts(
        # 是否显示标签
        is_show=False
    ),
)

# 配置
lines.set_global_opts(
    
    # 是否帖 y 轴
    xaxis_opts=opts.AxisOpts(
        type_="category", 
        boundary_gap=False
    ),
    
    # 图表标题
    title_opts=opts.TitleOpts(
        # 标题
        title=" 电池单体",
        # 标题位置
        pos_left="left",
        # 标题字体大小设置为 20
        title_textstyle_opts=opts.TextStyleOpts(font_size=20)
    ),
        
    # 图例设置
    legend_opts=opts.LegendOpts(
        # 显示图例，就是各个指标的切换按钮
        is_show=True, 
        # 图例设置为单选模式(multiple 多选)
        selected_mode='multiple',
        # 图例的位置
        pos_top='5%', pos_right='10%',
        # 图例的布局朝向，水平(vertical 垂直)
        orient='horizontal', 
        # 图例形状
        legend_icon='circle'
    ),
        
    # 工具箱设置
    toolbox_opts=opts.ToolboxOpts(
        # 是否显示工具栏组件
        is_show=True,
        # 布局朝向('vertical')
        orient='vertical',
        # 离上边距的距离
        pos_top='10%',
        # 离左边距的距离
        pos_left='91%',
            
        # 配置各个工具箱
        feature=opts.ToolBoxFeatureOpts(
                
            # 保存工具
            opts.ToolBoxFeatureSaveAsImageOpts(
                # 是否显示
                is_show=True,
                # 提示语
                title="保存",                 
            ),
                
            # 还原工具
            opts.ToolBoxFeatureRestoreOpts(
                # 是否显示该工具
                is_show=True,
                # 提示语
                title="还原", 
            ),
                
            # 数据视图工具
            opts.ToolBoxFeatureDataViewOpts(
                # 是否显示该工具
                is_show=True,
                # 提示语
                title="数据视图",
                # 是否不可编辑
                is_read_only=False,
            ),
            
            # 缩放工具配置项，直角坐标系适用
            opts.ToolBoxFeatureDataZoomOpts(
                # 是否显示该工具。
                is_show=True,
                # 提示语
                zoom_title="区域缩放",
                # 提示语
                back_title="缩放还原",
            ),
                
            # 图表类型切换，适用于直角坐标系
            opts.ToolBoxFeatureMagicTypeOpts(
                # 是否显示该工具
                is_show=True,
                # 启用的动态类型('stack','line','bar','tiled')
                type_=['stack','line','bar','tiled'],
                # 折线标题文本
                line_title="折线图",
                # 柱状标题文本
                bar_title="柱状图",
                # 堆积标题文本
                stack_title="堆叠",
                # 平铺标题文
                tiled_title="平铺",
            ),
                
            # 工具箱选框组件配置项
            opts.ToolBoxFeatureBrushOpts(  
                # 选择显示哪些选框
                type_=[]
            ),
        )
    ),
)
lines.render_notebook()

D:\programming\dev_env\anaconda\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


# 热力地图

## 运营的主要区域分布分析

展示每一天的车辆运行区域分布

分组计算，每一天每一个经纬度的数量

In [45]:
# 执行 SQL 并得到结果，转换为 DataFrame 类型
sql = '''
select 
	date_format(sendtime, "%Y-%m-%d") as time,
	latitude,
	longitude
from 
	activity a 
order by 
	time ;
'''
cur.execute(sql)
result = cur.fetchall()
result = pd.DataFrame(list(result),columns=['time','latitude','longitude'])

# 纬度，经度标准化，并计算每个每天每个经纬度出现次数
result['latitude'] = result['latitude'].map(lambda x: int(x / 1000000))
result['longitude'] = result['longitude'].map(lambda x: int(x / 1000000))
result = result.groupby(['time','latitude','longitude']).count().reset_index()

# 每一个经纬度打包
latitude = list(result['latitude'])
longitude = list(result['longitude'])
lat_lon = list(zip(latitude,longitude))

# 定位每一个经纬度的所属省份
# 调用腾讯地图 api
province = []
for i in lat_lon:
    url = 'https://apis.map.qq.com/ws/geocoder/v1/?location={},{}&key=43PBZ-PHLC6-OHTSQ-EUVQ6-PNI2V-KBBLJ'.format(
            i[0], i[1])
    response = requests.get(url)
    try:
        p = re.findall('"province": "(.*?)",', response.text, re.S)[0]
        province.append(p)
    except:
        province.append('异常')
        pass

In [48]:
# 把省份添加至经纬度对应的列，并剔除异常值
result['province'] = pd.DataFrame(province)
result = result[(result['province'] != '异常') & \
                (result['province'] != '中国区域')]

# 计算每一天每一个省份出现次数，作为阈值，值越大越活跃
result = result.groupby(['time','province']).count().reset_index()

# 踢掉'省
result['province'] = result['province'].str.replace('省','')
result = result[['time','province','latitude']]
result.rename(columns={'latitude' : '阈值'},inplace=True)

In [49]:
# 热力地图创建
# 以时间为切换依据，循环添加进图表
maps = Map(init_opts=opts.InitOpts(theme='dark')) 
for time in result['time'].unique():
    # 选择对应时间的省份
    data = result[result['time'] == time]
    
    # 添加值
    maps.add(time,
             [list(z) for z in zip(data['province'],data['阈值'])],
            'china')
    
    maps.set_global_opts(
        
        # 图表标题
        title_opts=opts.TitleOpts(
            # 标题
            title="地区活跃度阈值",
            # 标题位置
            pos_left="left",
            # 标题字体大小设置为 20
            title_textstyle_opts=opts.TextStyleOpts(font_size=20)
        ),
        
        # 视觉映射值设置
        visualmap_opts=opts.VisualMapOpts(
            # 最大值
            max_=13, 
            # 最小值
            min_=1,
            # 视觉映射显示
            is_show=True, 
            # 是否为分段形
            is_piecewise=True,
            # 组件映射维度
            dimension=0,
        ),
        
        # 图例设置
        legend_opts=opts.LegendOpts(
            # 显示图例，就是各个时间的切换按钮
            is_show=True, 
            # 图例设置为单选模式
            selected_mode='single',
            # 图例的位置
            pos_top='5%', pos_right='5%',
            # 图例的布局朝向，垂直
            orient='vertical', 
            # 图例形状
            legend_icon='circle'
        ),
        
        # 工具箱设置
        toolbox_opts=opts.ToolboxOpts(
            # 是否显示工具栏组件
            is_show=True,
            # 布局朝向
            orient='vertical',
            # 离上边距的距离
            pos_top='40%',
            pos_left='87%',
            
            # 配置各个工具箱
            feature=opts.ToolBoxFeatureOpts(
                
                # 保存工具
                opts.ToolBoxFeatureSaveAsImageOpts(
                    # 是否显示
                    is_show=True,
                    # 提示语
                    title="保存为图片",                 
                ),
                
                # 还原工具
                opts.ToolBoxFeatureRestoreOpts(
                    # 是否显示该工具
                    is_show=True,
                    # 提示语
                    title="还原", 
                ),
                
                # 数据视图工具
                opts.ToolBoxFeatureDataViewOpts(
                    # 是否显示该工具
                    is_show=True,
                    # 提示语
                    title="数据视图",
                    # 是否不可编辑
                    is_read_only=False,
                ),
                
                # 缩放工具配置项，直角坐标系适用
                opts.ToolBoxFeatureDataZoomOpts(
                    # 是否显示该工具。
                    is_show=False,
                    # 提示语
                    zoom_title="区域缩放",
                    # 提示语
                    back_title="区域缩放还原",
                ),
                
                # 图表类型切换，适用于直角坐标系
                opts.ToolBoxFeatureMagicTypeOpts(
                    # 是否显示该工具
                    is_show=False,
                    # 启用的动态类型
                    type_=['stack','line','bar','tiled'],                   
                ),
                
                # 工具箱选框组件配置项
                opts.ToolBoxFeatureBrushOpts(  
                    # 选择显示哪些选框
                    type_=[]
                ),
            )
        ),
    )
maps.render_notebook()

D:\programming\dev_env\anaconda\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


# 极坐标图

## 每一天车系分布

展示每一天车系占比分布

In [50]:
sql = '''
select
	date_format(sendtime, "%Y-%m-%d") as time,
	car_model ,
	count(distinct vin) as num
from
	activity a
group by
	date_format(sendtime, "%Y-%m-%d"),
	car_model
order by
	time;
'''
cur.execute(sql)
result = cur.fetchall()
result = pd.DataFrame(list(result),columns=['time','car_model','num'])

c = (
    Polar(init_opts=opts.InitOpts(theme='dark'))
    
    # 极坐标轴配置
    .add_schema(
        
        # 极坐标系角度轴配置项
        angleaxis_opts=opts.AngleAxisOpts(
            # 设置坐标轴
            data=list(result['time'].unique()),
            # 坐标轴方向，顺时针
            is_clockwise=True,            
        )
        
    )
    
    # 添加数据
    .add("东风品牌EJ04系列", 
         list(result[result['car_model'] == '东风品牌EJ04系列']['num']), 
         type_="bar", 
         stack="stack0")
    .add("东风品牌F15系列", 
         list(result[result['car_model'] == '东风品牌F15系列']['num']), 
         type_="bar", 
         stack="stack0")
    .add("东风品牌X37PHEV系列", 
         list(result[result['car_model'] == '东风品牌X37PHEV系列']['num']), 
         type_="bar", 
         stack="stack0")
    
    # 全局配置
    .set_global_opts(
        
        # 图表标题
        title_opts=opts.TitleOpts(
            # 标题
            title="车系分布",
            # 标题位置
            pos_left="left",
            # 标题字体大小设置为 20
            title_textstyle_opts=opts.TextStyleOpts(font_size=20)
        ),
        
        # 图例设置
        legend_opts=opts.LegendOpts(
            # 显示图例，就是各个时间的切换按钮
            is_show=True, 
            # 图例设置为多选模式(single 单选)
            selected_mode='multiple',
            # 图例的位置
            #pos_top='5%', pos_right='5%',
            # 图例的布局朝向，垂直('horizontal' 水平)
            #orient='vertical', 
            # 图例形状
            legend_icon='circle'
        ),
        
        # 工具箱设置
        toolbox_opts=opts.ToolboxOpts(
            # 是否显示工具栏组件
            is_show=True,
            # 布局朝向，水平(vertical 垂直)
            orient='vertical',
            # 离上边距的距离
            pos_top='30%',
            pos_left='86%',
            
            # 配置各个工具箱
            feature=opts.ToolBoxFeatureOpts(
                
                # 保存工具
                opts.ToolBoxFeatureSaveAsImageOpts(
                    # 是否显示
                    is_show=True,
                    # 提示语
                    title="保存为图片",                 
                ),
                
                # 还原工具
                opts.ToolBoxFeatureRestoreOpts(
                    # 是否显示该工具
                    is_show=True,
                    # 提示语
                    title="还原", 
                ),
                
                # 数据视图工具
                opts.ToolBoxFeatureDataViewOpts(
                    # 是否显示该工具
                    is_show=True,
                    # 提示语
                    title="数据视图",
                    # 是否不可编辑
                    is_read_only=False,
                ),
                
                # 缩放工具配置项，直角坐标系适用
                opts.ToolBoxFeatureDataZoomOpts(
                    # 是否显示该工具。
                    is_show=False,
                    # 提示语
                    zoom_title="区域缩放",
                    # 提示语
                    back_title="区域缩放还原",
                ),
                
                # 图表类型切换，适用于直角坐标系
                opts.ToolBoxFeatureMagicTypeOpts(
                    # 是否显示该工具
                    is_show=False,
                    # 启用的动态类型
                    type_=['stack','line','bar','tiled'],                   
                ),
                
                # 工具箱选框组件配置项
                opts.ToolBoxFeatureBrushOpts(
                    # 选择显示哪些选框
                    type_=[]
                ),
            )
        ),
    )
)
c.render_notebook()

D:\programming\dev_env\anaconda\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


## 运营中电压电池单体代号分布占比

展示电压电池单体代号每一天代号占比，代号以区间划分

In [51]:
sql = '''
select
	date_format(sendtime, "%Y-%m-%d") as time,
	max_V_bat_mer_code as '电压电池单体代号',
	count(max_V_bat_mer_code) as num
from 
	activity a
group by
	date_format(sendtime, "%Y-%m-%d"),
	max_V_bat_mer_code
order by 
	time;	
'''
cur.execute(sql)
result = cur.fetchall()
result = pd.DataFrame(list(result),columns=['time','电压电池单体代号','num'])

# 代号分区
result['电压电池单体代号'] = pd.cut(result['电压电池单体代号'],
                            bins=[0,20,40,60,80,100])
result['电压电池单体代号'] = result['电压电池单体代号'].astype(str)

# 分组计算每个区间代号的总和
result = result.groupby(['time','电压电池单体代号']).sum().reset_index()

c = (
    Polar(init_opts=opts.InitOpts(theme='dark'))
    
    # 极坐标轴配置
    .add_schema(
        
        # 极坐标系角度轴配置项
        angleaxis_opts=opts.AngleAxisOpts(
            # 设置坐标轴
            data=list(result['time'].unique()),
            # 坐标轴方向，顺时针
            is_clockwise=True,            
        )
        
    )
    
    # 添加数据
    .add("(0, 20]", 
         list(result[result['电压电池单体代号'] == '(0, 20]']['num']), 
         type_="bar", 
         stack="stack0")
    .add("(20, 40]", 
         list(result[result['电压电池单体代号'] == '(20, 40]']['num']), 
         type_="bar", 
         stack="stack0")
    .add("(40, 60]", 
         list(result[result['电压电池单体代号'] == '(40, 60]']['num']), 
         type_="bar", 
         stack="stack0")
    .add("(60, 80]", 
         list(result[result['电压电池单体代号'] == '(60, 80]']['num']), 
         type_="bar", 
         stack="stack0")
    .add("(80, 100]", 
         list(result[result['电压电池单体代号'] == '(80, 100]']['num']), 
         type_="bar", 
         stack="stack0")
    
    # 全局配置
    .set_global_opts(
        
        # 图表标题
        title_opts=opts.TitleOpts(
            # 标题
            title="电池单体代号分布",
            # 标题位置
            pos_left="left",
            # 标题字体大小设置为 20
            title_textstyle_opts=opts.TextStyleOpts(font_size=20)
        ),
        
        # 图例设置
        legend_opts=opts.LegendOpts(
            # 显示图例，就是各个时间的切换按钮
            is_show=True, 
            # 图例设置为多选模式(single 单选)
            selected_mode='multiple',
            # 图例的位置
            pos_top='5%', pos_right='5%',
            # 图例的布局朝向，垂直('horizontal' 水平)
            orient='vertical', 
            # 图例形状
            legend_icon='circle'
        ),
        
        # 工具箱设置
        toolbox_opts=opts.ToolboxOpts(
            # 是否显示工具栏组件
            is_show=True,
            # 布局朝向，水平(vertical 垂直)
            orient='vertical',
            # 离上边距的距离
            pos_top='30%',
            pos_left='86%',
            
            # 配置各个工具箱
            feature=opts.ToolBoxFeatureOpts(
                
                # 保存工具
                opts.ToolBoxFeatureSaveAsImageOpts(
                    # 是否显示
                    is_show=True,
                    # 提示语
                    title="保存为图片",                 
                ),
                
                # 还原工具
                opts.ToolBoxFeatureRestoreOpts(
                    # 是否显示该工具
                    is_show=True,
                    # 提示语
                    title="还原", 
                ),
                
                # 数据视图工具
                opts.ToolBoxFeatureDataViewOpts(
                    # 是否显示该工具
                    is_show=True,
                    # 提示语
                    title="数据视图",
                    # 是否不可编辑
                    is_read_only=False,
                ),
                
                # 缩放工具配置项，直角坐标系适用
                opts.ToolBoxFeatureDataZoomOpts(
                    # 是否显示该工具。
                    is_show=False,
                    # 提示语
                    zoom_title="区域缩放",
                    # 提示语
                    back_title="区域缩放还原",
                ),
                
                # 图表类型切换，适用于直角坐标系
                opts.ToolBoxFeatureMagicTypeOpts(
                    # 是否显示该工具
                    is_show=False,
                    # 启用的动态类型
                    type_=['stack','line','bar','tiled'],                   
                ),
                
                # 工具箱选框组件配置项
                opts.ToolBoxFeatureBrushOpts(
                    # 选择显示哪些选框
                    type_=[]
                ),
            )
        ),
    )
)
c.render_notebook()

D:\programming\dev_env\anaconda\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


## 运营中温度探针序号分布

温度探针序号每一天分布，以区间划分序号

In [52]:
sql = '''
select
	date_format(sendtime, "%Y-%m-%d") as time,
	max_TEMP_probe_serial as '温度探针序号',
	count(max_TEMP_probe_serial) as num
from 
	activity a
group by
	date_format(sendtime, "%Y-%m-%d"),
	max_TEMP_probe_serial
order by 
	time;	
'''
cur.execute(sql)
result = cur.fetchall()
result = pd.DataFrame(list(result),columns=['time','温度探针序号','num'])

# 代号分区
result['温度探针序号'] = pd.cut(result['温度探针序号'],
                            bins=[0,20,40,60])
result['温度探针序号'] = result['温度探针序号'].astype(str)

# 分组计算每个区间代号的总和
result = result.groupby(['time','温度探针序号']).sum().reset_index()

c = (
    Polar(init_opts=opts.InitOpts(theme='dark'))
    
    # 极坐标轴配置
    .add_schema(
        
        # 极坐标系角度轴配置项
        angleaxis_opts=opts.AngleAxisOpts(
            # 设置坐标轴
            data=list(result['time'].unique()),
            # 坐标轴方向，顺时针
            is_clockwise=True,            
        )
        
    )
    
    # 添加数据
    .add("(0, 20]", 
         list(result[result['温度探针序号'] == '(0, 20]']['num']), 
         type_="bar", 
         stack="stack0")
    .add("(20, 40]", 
         list(result[result['温度探针序号'] == '(20, 40]']['num']), 
         type_="bar", 
         stack="stack0")
    .add("(40, 60]", 
         list(result[result['温度探针序号'] == '(40, 60]']['num']), 
         type_="bar", 
         stack="stack0")
    
    # 全局配置
    .set_global_opts(
        
        # 图表标题
        title_opts=opts.TitleOpts(
            # 标题
            title="温度探针序号分布",
            # 标题位置
            pos_left="left",
            # 标题字体大小设置为 20
            title_textstyle_opts=opts.TextStyleOpts(font_size=20)
        ),
        
        # 图例设置
        legend_opts=opts.LegendOpts(
            # 显示图例，就是各个时间的切换按钮
            is_show=True, 
            # 图例设置为多选模式(single 单选)
            selected_mode='multiple',
            # 图例的位置
            pos_top='5%', pos_right='5%',
            # 图例的布局朝向，垂直('horizontal' 水平)
            orient='vertical', 
            # 图例形状
            legend_icon='circle'
        ),
        
        # 工具箱设置
        toolbox_opts=opts.ToolboxOpts(
            # 是否显示工具栏组件
            is_show=True,
            # 布局朝向，水平(vertical 垂直)
            orient='vertical',
            # 离上边距的距离
            pos_top='30%',
            pos_left='86%',
            
            # 配置各个工具箱
            feature=opts.ToolBoxFeatureOpts(
                
                # 保存工具
                opts.ToolBoxFeatureSaveAsImageOpts(
                    # 是否显示
                    is_show=True,
                    # 提示语
                    title="保存为图片",                 
                ),
                
                # 还原工具
                opts.ToolBoxFeatureRestoreOpts(
                    # 是否显示该工具
                    is_show=True,
                    # 提示语
                    title="还原", 
                ),
                
                # 数据视图工具
                opts.ToolBoxFeatureDataViewOpts(
                    # 是否显示该工具
                    is_show=True,
                    # 提示语
                    title="数据视图",
                    # 是否不可编辑
                    is_read_only=False,
                ),
                
                # 缩放工具配置项，直角坐标系适用
                opts.ToolBoxFeatureDataZoomOpts(
                    # 是否显示该工具。
                    is_show=False,
                    # 提示语
                    zoom_title="区域缩放",
                    # 提示语
                    back_title="区域缩放还原",
                ),
                
                # 图表类型切换，适用于直角坐标系
                opts.ToolBoxFeatureMagicTypeOpts(
                    # 是否显示该工具
                    is_show=False,
                    # 启用的动态类型
                    type_=['stack','line','bar','tiled'],                   
                ),
                
                # 工具箱选框组件配置项
                opts.ToolBoxFeatureBrushOpts(
                    # 选择显示哪些选框
                    type_=[]
                ),
            )
        ),
    )
)
c.render_notebook()

D:\programming\dev_env\anaconda\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


## 运营中可用单体电池总数分布占比

每一天可用单体电池总数分布占比

In [53]:
sql = '''
select
	date_format(sendtime, "%Y-%m-%d") as time,
	total_mer_battery,
	count(total_mer_battery) as num
from 
	activity a
group by
	date_format(sendtime, "%Y-%m-%d"),
	total_mer_battery
order by 
	time;		
'''
cur.execute(sql)
result = cur.fetchall()
result = pd.DataFrame(list(result),columns=['time','total_mer_battery','num'])

c = (
    Polar(init_opts=opts.InitOpts(theme='dark'))
    
    # 极坐标轴配置
    .add_schema(
        
        # 极坐标系角度轴配置项
        angleaxis_opts=opts.AngleAxisOpts(
            # 设置坐标轴
            data=list(result['time'].unique()),
            # 坐标轴方向，顺时针
            is_clockwise=True,            
        )
        
    )
    
    # 添加数据
    .add("84", 
         list(result[result['total_mer_battery'] == 84]['num']), 
         type_="bar", 
         stack="stack0")
    .add("94", 
         list(result[result['total_mer_battery'] == 94]['num']), 
         type_="bar", 
         stack="stack0")
    .add("96", 
         list(result[result['total_mer_battery'] == 96]['num']), 
         type_="bar", 
         stack="stack0")
    
    # 全局配置
    .set_global_opts(
        
        # 图表标题
        title_opts=opts.TitleOpts(
            # 标题
            title="温度探针序号分布",
            # 标题位置
            pos_left="left",
            # 标题字体大小设置为 20
            title_textstyle_opts=opts.TextStyleOpts(font_size=20)
        ),
        
        # 图例设置
        legend_opts=opts.LegendOpts(
            # 显示图例，就是各个时间的切换按钮
            is_show=True, 
            # 图例设置为多选模式(single 单选)
            selected_mode='multiple',
            # 图例的位置
            pos_top='5%', pos_right='5%',
            # 图例的布局朝向，垂直('horizontal' 水平)
            orient='vertical', 
            # 图例形状
            legend_icon='circle'
        ),
        
        # 工具箱设置
        toolbox_opts=opts.ToolboxOpts(
            # 是否显示工具栏组件
            is_show=True,
            # 布局朝向，水平(vertical 垂直)
            orient='vertical',
            # 离上边距的距离
            pos_top='30%',
            pos_left='86%',
            
            # 配置各个工具箱
            feature=opts.ToolBoxFeatureOpts(
                
                # 保存工具
                opts.ToolBoxFeatureSaveAsImageOpts(
                    # 是否显示
                    is_show=True,
                    # 提示语
                    title="保存为图片",                 
                ),
                
                # 还原工具
                opts.ToolBoxFeatureRestoreOpts(
                    # 是否显示该工具
                    is_show=True,
                    # 提示语
                    title="还原", 
                ),
                
                # 数据视图工具
                opts.ToolBoxFeatureDataViewOpts(
                    # 是否显示该工具
                    is_show=True,
                    # 提示语
                    title="数据视图",
                    # 是否不可编辑
                    is_read_only=False,
                ),
                
                # 缩放工具配置项，直角坐标系适用
                opts.ToolBoxFeatureDataZoomOpts(
                    # 是否显示该工具。
                    is_show=False,
                    # 提示语
                    zoom_title="区域缩放",
                    # 提示语
                    back_title="区域缩放还原",
                ),
                
                # 图表类型切换，适用于直角坐标系
                opts.ToolBoxFeatureMagicTypeOpts(
                    # 是否显示该工具
                    is_show=False,
                    # 启用的动态类型
                    type_=['stack','line','bar','tiled'],                   
                ),
                
                # 工具箱选框组件配置项
                opts.ToolBoxFeatureBrushOpts(
                    # 选择显示哪些选框
                    type_=[]
                ),
            )
        ),
    )
)
c.render_notebook()

D:\programming\dev_env\anaconda\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


# 条形图

## 车辆出行类型 

展示每一天各报警的车辆数

In [54]:
sql = '''
select 
	date_format(sendtime, "%Y-%m-%d") as time,
	count(distinct vin,case when car_modeltype = 'DFM7000G1F3BEV' then car_modeltype end) as '类型1',
	count(distinct vin,case when car_modeltype = 'DFM7000G1F6BEV' then car_modeltype end) as '类型2',
	count(distinct vin,case when car_modeltype = 'DFM7000G1F2BEV' then car_modeltype end) as '类型3',
	count(distinct vin,case when car_modeltype = 'DFM7000H2DBEV' then car_modeltype end) as '类型4',
	count(distinct vin,case when car_modeltype = 'DFM7000H2ABEV1' then car_modeltype end) as '类型5',
	count(distinct vin,case when car_modeltype = 'DFM7000H2DBEV1' then car_modeltype end) as '类型6',
	count(distinct vin,case when car_modeltype = 'DFM7000G1F7BEV' then car_modeltype end) as '类型7',
	count(distinct vin,case when car_modeltype = 'DFM7000G1ABEV' then car_modeltype end) as '类型8',
	count(distinct vin,case when car_modeltype = 'DFM7000H2ABEV' then car_modeltype end) as '类型9',
	count(distinct vin,case when car_modeltype = 'DFM7000G1BBEV' then car_modeltype end) as '类型10',
	count(distinct vin,case when car_modeltype = 'DFM6460D5F1CHEV' then car_modeltype end) as '类型11',
	count(distinct vin,case when car_modeltype = 'DFM7000G1ABEV1' then car_modeltype end) as '类型12'
from 
	activity a 
group by
	date_format(sendtime, "%Y-%m-%d")
order by 
	time ;		
'''
cur.execute(sql)
result = cur.fetchall()
result = pd.DataFrame(list(result),columns=['time','类型1','类型2','类型3','类型4','类型5','类型6','类型7','类型8','类型9','类型10','类型11','类型12'])

# 柱状图创建
bars = Bar(init_opts=opts.InitOpts(theme='dark')) 

# 添加值
bars.add_xaxis(list(result['time'])),
bars.add_yaxis("类型1", list(result['类型1'])),
bars.add_yaxis("类型2", list(result['类型2'])),
bars.add_yaxis("类型3", list(result['类型3'])),
bars.add_yaxis("类型4", list(result['类型4'])),
bars.add_yaxis("类型5", list(result['类型5'])),
bars.add_yaxis("类型6", list(result['类型6'])),
bars.add_yaxis("类型7", list(result['类型7'])),
bars.add_yaxis("类型8", list(result['类型8'])),
bars.add_yaxis("类型9", list(result['类型9'])),
bars.add_yaxis("类型10", list(result['类型10'])),
bars.add_yaxis("类型11", list(result['类型11'])),
bars.add_yaxis("类型12", list(result['类型12'])),

# 翻转 xy 轴
bars.reversal_axis()

# 系列配置
bars.set_series_opts(
    
    # 标签配置
    label_opts=opts.LabelOpts(
        # 标签位置
        position="right"
    )
)

# 全局配置
bars.set_global_opts(
        
    # 图表标题
    title_opts=opts.TitleOpts(
        # 标题
        title="车辆出行类型 ",
        # 标题位置
        pos_left="left",
        # 标题字体大小设置为 20
        title_textstyle_opts=opts.TextStyleOpts(font_size=20)
    ),
        
    # 图例设置
    legend_opts=opts.LegendOpts(
        # 显示图例，就是各个指标的切换按钮
        is_show=True, 
        # 图例设置为单选模式(multiple 多选)
        selected_mode='multiple',
        # 图例的位置
        pos_top='5%',# pos_right='50%',
        # 图例的布局朝向，水平(vertical 垂直)
        orient='horizontal', 
        # 图例形状
        legend_icon='circle'
    ),
        
    # 工具箱设置
    toolbox_opts=opts.ToolboxOpts(
        # 是否显示工具栏组件
        is_show=True,
        # 布局朝向('vertical')
        orient='vertical',
        # 离上边距的距离
        pos_top='10%',
        # 离左边距的距离
        pos_left='91%',
            
        # 配置各个工具箱
        feature=opts.ToolBoxFeatureOpts(
                
            # 保存工具
            opts.ToolBoxFeatureSaveAsImageOpts(
                # 是否显示
                is_show=True,
                # 提示语
                title="保存",                 
            ),
                
            # 还原工具
            opts.ToolBoxFeatureRestoreOpts(
                # 是否显示该工具
                is_show=True,
                # 提示语
                title="还原", 
            ),
                
            # 数据视图工具
            opts.ToolBoxFeatureDataViewOpts(
                # 是否显示该工具
                is_show=True,
                # 提示语
                title="数据视图",
                # 是否不可编辑
                is_read_only=False,
            ),
            
            # 缩放工具配置项，直角坐标系适用
            opts.ToolBoxFeatureDataZoomOpts(
                # 是否显示该工具。
                is_show=True,
                # 提示语
                zoom_title="区域缩放",
                # 提示语
                back_title="缩放还原",
            ),
                
            # 图表类型切换，适用于直角坐标系
            opts.ToolBoxFeatureMagicTypeOpts(
                # 是否显示该工具
                is_show=True,
                # 启用的动态类型('stack','line','bar','tiled')
                type_=['stack','line','bar','tiled'],
                # 折线标题文本
                line_title="折线图",
                # 柱状标题文本
                bar_title="柱状图",
                # 堆积标题文本
                stack_title="堆叠",
                # 平铺标题文
                tiled_title="平铺",
            ),
                
            # 工具箱选框组件配置项
            opts.ToolBoxFeatureBrushOpts(  
                # 选择显示哪些选框
                type_=[]
            ),
        )
    ),
)
bars.render_notebook()

D:\programming\dev_env\anaconda\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


# 仪表盘

In [56]:
import pymysql
import requests
import re
import pandas as pd 
import numpy as np 
import random
from pyecharts import options as opts
from pyecharts.charts import Map 
from pyecharts.charts import Line
from pyecharts.charts import Polar
from pyecharts.charts import Bar
from pyecharts.charts import Page


# 连接数据库
conn = pymysql.connect(host='localhost',user='root',password='123456',
                       database='activity_analysic',port=3306,charset='gbk')
#创建游标
cur = conn.cursor()

def heat_map():
    '''
    展示每一天的车辆运行区域分布
    '''
    # 执行 SQL 并得到结果，转换为 DataFrame 类型
    sql = '''
    select 
        date_format(sendtime, "%Y-%m-%d") as time,
        latitude,
        longitude
    from 
        activity a 
    order by 
        time ;
    '''
    cur.execute(sql)
    result = cur.fetchall()
    result = pd.DataFrame(list(result),columns=['time','latitude','longitude'])

    # 纬度，经度标准化，并计算每个每天每个经纬度出现次数
    result['latitude'] = result['latitude'].map(lambda x: int(x / 1000000))
    result['longitude'] = result['longitude'].map(lambda x: int(x / 1000000))
    result = result.groupby(['time','latitude','longitude']).count().reset_index()

    # 每一个经纬度打包
    latitude = list(result['latitude'])
    longitude = list(result['longitude'])
    lat_lon = list(zip(latitude,longitude))

    # 定位每一个经纬度的所属省份
    province = []
    for i in lat_lon:
        url = 'https://apis.map.qq.com/ws/geocoder/v1/?location={},{}&key=43PBZ-PHLC6-OHTSQ-EUVQ6-PNI2V-KBBLJ'.format(
                i[0], i[1])
        response = requests.get(url)
        try:
            p = re.findall('"province": "(.*?)",', response.text, re.S)[0]
            province.append(p)
        except:
            province.append('异常')
            pass
        
    # 把省份添加至经纬度对应的列，并剔除异常值
    result['province'] = pd.DataFrame(province)
    result = result[(result['province'] != '异常') & \
                    (result['province'] != '中国区域')]

    # 计算每一天每一个省份出现次数，作为阈值，值越大越活跃
    result = result.groupby(['time','province']).count().reset_index()

    # 踢掉'省
    result['province'] = result['province'].str.replace('省','')
    result = result[['time','province','latitude']]
    result.rename(columns={'latitude' : '阈值'},inplace=True)
    
    # 热力地图创建
    # 以时间为切换依据，循环添加进图表
    maps = Map(init_opts=opts.InitOpts(theme='dark',chart_id=1)) 
    for time in result['time'].unique():
        # 选择对应时间的省份
        data = result[result['time'] == time]

        # 添加值
        maps.add(time,
                 [list(z) for z in zip(data['province'],data['阈值'])],
                'china')

        maps.set_global_opts(

            # 图表标题
            title_opts=opts.TitleOpts(
                # 标题
                title="地区活跃度阈值",
                # 标题位置
                pos_left="left",
                # 标题字体大小设置为 20
                title_textstyle_opts=opts.TextStyleOpts(font_size=20)
            ),

            # 视觉映射值设置
            visualmap_opts=opts.VisualMapOpts(
                # 最大值
                max_=13, 
                # 最小值
                min_=1,
                # 视觉映射显示
                is_show=True, 
                # 是否为分段形
                is_piecewise=True,
                # 组件映射维度
                dimension=0,
            ),

            # 图例设置
            legend_opts=opts.LegendOpts(
                # 显示图例，就是各个时间的切换按钮
                is_show=True, 
                # 图例设置为单选模式
                selected_mode='single',
                # 图例的位置
                pos_top='5%', pos_right='5%',
                # 图例的布局朝向，垂直
                orient='vertical', 
                # 图例形状
                legend_icon='circle'
            ),

            # 工具箱设置
            toolbox_opts=opts.ToolboxOpts(
                # 是否显示工具栏组件
                is_show=True,
                # 布局朝向
                orient='vertical',
                # 离上边距的距离
                pos_top='40%',
                pos_left='87%',

                # 配置各个工具箱
                feature=opts.ToolBoxFeatureOpts(

                    # 保存工具
                    opts.ToolBoxFeatureSaveAsImageOpts(
                        # 是否显示
                        is_show=True,
                        # 提示语
                        title="保存为图片",                 
                    ),

                    # 还原工具
                    opts.ToolBoxFeatureRestoreOpts(
                        # 是否显示该工具
                        is_show=True,
                        # 提示语
                        title="还原", 
                    ),

                    # 数据视图工具
                    opts.ToolBoxFeatureDataViewOpts(
                        # 是否显示该工具
                        is_show=True,
                        # 提示语
                        title="数据视图",
                        # 是否不可编辑
                        is_read_only=False,
                    ),

                    # 缩放工具配置项，直角坐标系适用
                    opts.ToolBoxFeatureDataZoomOpts(
                        # 是否显示该工具。
                        is_show=False,
                        # 提示语
                        zoom_title="区域缩放",
                        # 提示语
                        back_title="区域缩放还原",
                    ),

                    # 图表类型切换，适用于直角坐标系
                    opts.ToolBoxFeatureMagicTypeOpts(
                        # 是否显示该工具
                        is_show=False,
                        # 启用的动态类型
                        type_=['stack','line','bar','tiled'],                   
                    ),

                    # 工具箱选框组件配置项
                    opts.ToolBoxFeatureBrushOpts(  
                        # 选择显示哪些选框
                        type_=[]
                    ),
                )
            ),
        )
    return maps
    

def bar():
    '''
    展示每一天车辆出行平均车速的车辆统计分布
    每一天车辆的 SOC 平均取值
    可充电储能温度探针个数每一天占比
    展示每一天各充电状态的车辆总数
    '''
    # 出行平均车速分布
    sql1 = '''
    select 
        date_format(sendtime, "%Y-%m-%d") as time,
        avg(speed) avg_speed
    from 
        activity a 
    group by
        date_format(sendtime, "%Y-%m-%d")
    order by 
        time ;
    '''
    cur.execute(sql1)
    result1 = cur.fetchall()
    result1 = pd.DataFrame(list(result1),columns=['time','平均车速'])

    # SOC 变化状态
    sql2 = '''
    select 
        date_format(sendtime, "%Y-%m-%d") as time,
        avg(SOC) avg_SOC
    from 
        activity a 
    group by
        date_format(sendtime, "%Y-%m-%d")
    order by 
        time ;
    '''
    cur.execute(sql2)
    result2 = cur.fetchall()
    result2 = pd.DataFrame(list(result2),columns=['time','平均SOC'])

    sql3 = '''
    select
        date_format(sendtime, "%Y-%m-%d") as time,
        count(total_TEMP_probe) as '可充电储能温度探针个数均值'
    from 
        activity a
    group by
        date_format(sendtime, "%Y-%m-%d")
    order by 
        time;	
    '''
    cur.execute(sql3)
    result3 = cur.fetchall()
    result3 = pd.DataFrame(list(result3),columns=['time','可充电储能温度探针个数均值'])
    result3['可充电储能温度探针个数均值'] = result3['可充电储能温度探针个数均值']/100

    # 出行平均车速分布
    sql4 = '''
    with charge_state_tb as(
        select 
            date_format(sendtime, "%Y-%m-%d") as time,
            charge_state,
            count(distinct vin) as num
        from 
            activity 
        group by
            date_format(sendtime, "%Y-%m-%d"),charge_state
        order by 
            time
    )
    select 
        a.time,
        a.num '停车充电',
        b.num  '行驶充电',
        c.num '未充电状态',
        d.num '充电完成'
    from 
        (
            select 
                time,num 
            from 
                charge_state_tb 
            where
                charge_state = 1
        ) a 
    join 
        (
            select 
                time,num
            from 
                charge_state_tb
            where
                charge_state = 2
        ) b
    on 
        a.time = b.time
    join 
        (
            select 
                time,num
            from 
                charge_state_tb
            where
                charge_state = 3
        ) c
    on 
        a.time = c.time
    join 
        (
            select 
                time,num
            from 
                charge_state_tb
            where
                charge_state = 4
        ) d
    on 
        a.time = d.time
    '''
    cur.execute(sql4)
    result4 = cur.fetchall()
    result4 = pd.DataFrame(list(result4),columns=['time','停车充电','行驶充电','未充电状态','充电完成'])
    result4['行驶充电'] = result4['行驶充电'] * 50
    result4['充电完成'] = result4['充电完成'] * 10

    # 合并三张表
    result = pd.merge(result1,result2,on='time')
    result = pd.merge(result,result3,on='time')
    result = pd.merge(result,result4,on='time')

    # 柱状图创建
    bars = Bar(init_opts=opts.InitOpts(theme='dark',chart_id=2)) 

    # 添加值
    bars.add_xaxis(list(result['time'])),
    bars.add_yaxis("平均车速", list(result['平均车速'])),
    bars.add_yaxis("平均SOC", list(result['平均SOC'])),
    bars.add_yaxis("可充电储能温度探针个数均值", list(result['可充电储能温度探针个数均值'])),
    bars.add_yaxis("停车充电", list(result['停车充电'])),
    bars.add_yaxis("行驶充电", list(result['行驶充电'])),
    bars.add_yaxis("未充电状态", list(result['未充电状态'])),
    bars.add_yaxis("充电完成", list(result['充电完成'])),

    # 系列配置
    bars.set_series_opts(

        # 标签
        label_opts=opts.LabelOpts(
            # 是否显示标签
            is_show=False
        ),
    )

    # 全局配置
    bars.set_global_opts(

        # 图表标题
        title_opts=opts.TitleOpts(
            # 标题
            title="车速/SOC/探针数/充电状态",
            # 标题位置
            pos_left="left",
            # 标题字体大小设置为 20
            title_textstyle_opts=opts.TextStyleOpts(font_size=15)
        ),

        # 图例设置
        legend_opts=opts.LegendOpts(
            # 显示图例，就是各个指标的切换按钮
            is_show=True, 
            # 图例设置为单选模式(multiple 多选)
            selected_mode='multiple',
            # 图例的位置
            pos_top='5%', pos_right='10%',
            # 图例的布局朝向，水平(vertical 垂直)
            orient='horizontal', 
            # 图例形状
            legend_icon='circle'
        ),

        # 工具箱设置
        toolbox_opts=opts.ToolboxOpts(
            # 是否显示工具栏组件
            is_show=True,
            # 布局朝向('vertical')
            orient='vertical',
            # 离上边距的距离
            pos_top='10%',
            # 离左边距的距离
            pos_left='91%',

            # 配置各个工具箱
            feature=opts.ToolBoxFeatureOpts(

                # 保存工具
                opts.ToolBoxFeatureSaveAsImageOpts(
                    # 是否显示
                    is_show=True,
                    # 提示语
                    title="保存",                 
                ),

                # 还原工具
                opts.ToolBoxFeatureRestoreOpts(
                    # 是否显示该工具
                    is_show=True,
                    # 提示语
                    title="还原", 
                ),

                # 数据视图工具
                opts.ToolBoxFeatureDataViewOpts(
                    # 是否显示该工具
                    is_show=True,
                    # 提示语
                    title="数据视图",
                    # 是否不可编辑
                    is_read_only=False,
                ),

                # 缩放工具配置项，直角坐标系适用
                opts.ToolBoxFeatureDataZoomOpts(
                    # 是否显示该工具。
                    is_show=True,
                    # 提示语
                    zoom_title="区域缩放",
                    # 提示语
                    back_title="缩放还原",
                ),

                # 图表类型切换，适用于直角坐标系
                opts.ToolBoxFeatureMagicTypeOpts(
                    # 是否显示该工具
                    is_show=True,
                    # 启用的动态类型('stack','line','bar','tiled')
                    type_=['stack','line','bar','tiled'],
                    # 折线标题文本
                    line_title="折线图",
                    # 柱状标题文本
                    bar_title="柱状图",
                    # 堆积标题文本
                    stack_title="堆叠",
                    # 平铺标题文
                    tiled_title="平铺",
                ),

                # 工具箱选框组件配置项
                opts.ToolBoxFeatureBrushOpts(  
                    # 选择显示哪些选框
                    type_=[]
                ),
            )
        ),
    )
    return bars 
    
    
def stack_bar():
    '''
    展示每一天的车辆状态为”车辆启动状态”的车辆数
    统计每一天运行模式
    展示每一天各种驱动机状态总数
    '''
    # 车辆状态
    sql1 = '''
    with vehicle_status_tb as(
        select 
            date_format(sendtime, "%Y-%m-%d") as time,
            vehicle_status,
            count(distinct vin) as num
        from 
            activity 
        group by
            date_format(sendtime, "%Y-%m-%d"),vehicle_status
        order by 
            time
    )
    select 
        a.time,
        a.num '车辆启动状态',
        b.num  '熄火',
        c.num '其他状态'
    from 
        (
            select 
                time,num 
            from 
                vehicle_status_tb 
            where
                vehicle_status = 1
        ) a 
    join 
        (
            select 
                time,num
            from 
                vehicle_status_tb 
            where
                vehicle_status = 2
        ) b
    on 
        a.time = b.time
    join 
        (
            select 
                time,num
            from 
                vehicle_status_tb 
            where
                vehicle_status = 3
        ) c
    on 
        a.time = c.time
    '''
    cur.execute(sql1)
    result1 = cur.fetchall()
    result1 = pd.DataFrame(list(result1),columns=['time','启动状态','熄火状态','其他状态'])

    # 车辆模式
    sql2 = '''
    with run_model_tb as(
        select 
            date_format(sendtime, "%Y-%m-%d") as time,
            run_model,
            count(distinct vin) as num
        from 
            activity 
        group by
            date_format(sendtime, "%Y-%m-%d"),run_model
        order by 
            time
    )
    select 
        a.time,
        a.num '纯点',
        b.num  '混动'
    from 
        (
            select 
                time,num 
            from 
                run_model_tb
            where
                run_model = 1
        ) a 
    join 
        (
            select 
                time,num
            from 
                run_model_tb
            where
                run_model = 2
        ) b
    on 
        a.time = b.time
    '''
    cur.execute(sql2)
    result2 = cur.fetchall()
    result2 = pd.DataFrame(list(result2),columns=['time','纯点','混动'])

    # 驱动机
    sql3 = '''
    with dm_state_tb as(
        select 
            date_format(sendtime, "%Y-%m-%d") as time,
            DM_status,
            count(distinct vin) as num
        from 
            activity 
        group by
            date_format(sendtime, "%Y-%m-%d"),DM_status
        order by 
            time
    )
    select 
        a.time,
        a.num '耗电',
        b.num  '发电',
        c.num '关闭状态',
        d.num '准备状态'
    from 
        (
            select 
                time,num 
            from 
                dm_state_tb 
            where
                DM_status = 1
        ) a 
    join 
        (
            select 
                time,num
            from 
                dm_state_tb 
            where
                DM_status = 2
        ) b
    on 
        a.time = b.time
    join 
        (
            select 
                time,num
            from 
                dm_state_tb 
            where
                DM_status = 3
        ) c
    on 
        a.time = c.time
    join 
        (
            select 
                time,num
            from 
                dm_state_tb 
            where
                DM_status = 4
        ) d
    on 
        a.time = d.time
    '''
    cur.execute(sql3)
    result3 = cur.fetchall()
    result3 = pd.DataFrame(list(result3),columns=['time','耗电','发电','关闭状态',
                                                '准备状态'])

    # 合并表
    result = pd.merge(result1,result2,on='time')
    result = pd.merge(result,result3,on='time')

    # 柱状图创建
    bars = Bar(init_opts=opts.InitOpts(theme='dark',chart_id=3)) 

    # 添加值
    bars.add_xaxis(list(result['time'])),
    bars.add_yaxis("启动状态", list(result['启动状态']),stack="stack1"),
    bars.add_yaxis("熄火状态", list(result['熄火状态']),stack="stack1"),
    bars.add_yaxis("其他状态", list(result['其他状态']),stack="stack1"),
    bars.add_yaxis("纯点模式", list(result['纯点']),stack="stack1"),
    bars.add_yaxis("混动模式", list(result['混动']),stack="stack1"),
    bars.add_yaxis("耗电状态", list(result['耗电']),stack="stack1"),
    bars.add_yaxis("发电状态", list(result['发电']),stack="stack1"),
    bars.add_yaxis("关闭状态", list(result['关闭状态']),stack="stack1"),
    bars.add_yaxis("准备状态", list(result['准备状态']),stack="stack1"),

    # 配置
    bars.set_global_opts(

        # 图表标题
        title_opts=opts.TitleOpts(
            # 标题
            title="车辆状态/模式",
            # 标题位置
            pos_left="left",
            # 标题字体大小设置为 20
            title_textstyle_opts=opts.TextStyleOpts(font_size=20)
        ),

        # 图例设置
        legend_opts=opts.LegendOpts(
            # 显示图例，就是各个指标的切换按钮
            is_show=True, 
            # 图例设置为单选模式(multiple 多选)
            selected_mode='multiple',
            # 图例的位置
            pos_top='5%', pos_right='10%',
            # 图例的布局朝向，水平(vertical 垂直)
            orient='horizontal', 
            # 图例形状
            legend_icon='circle'
        ),

        # 工具箱设置
        toolbox_opts=opts.ToolboxOpts(
            # 是否显示工具栏组件
            is_show=True,
            # 布局朝向('vertical')
            orient='vertical',
            # 离上边距的距离
            pos_top='10%',
            # 离左边距的距离
            pos_left='91%',

            # 配置各个工具箱
            feature=opts.ToolBoxFeatureOpts(

                # 保存工具
                opts.ToolBoxFeatureSaveAsImageOpts(
                    # 是否显示
                    is_show=True,
                    # 提示语
                    title="保存",                 
                ),

                # 还原工具
                opts.ToolBoxFeatureRestoreOpts(
                    # 是否显示该工具
                    is_show=True,
                    # 提示语
                    title="还原", 
                ),

                # 数据视图工具
                opts.ToolBoxFeatureDataViewOpts(
                    # 是否显示该工具
                    is_show=True,
                    # 提示语
                    title="数据视图",
                    # 是否不可编辑
                    is_read_only=False,
                ),

                # 缩放工具配置项，直角坐标系适用
                opts.ToolBoxFeatureDataZoomOpts(
                    # 是否显示该工具。
                    is_show=True,
                    # 提示语
                    zoom_title="区域缩放",
                    # 提示语
                    back_title="缩放还原",
                ),

                # 图表类型切换，适用于直角坐标系
                opts.ToolBoxFeatureMagicTypeOpts(
                    # 是否显示该工具
                    is_show=True,
                    # 启用的动态类型('stack','line','bar','tiled')
                    type_=['stack','line','bar','tiled'],
                    # 折线标题文本
                    line_title="折线图",
                    # 柱状标题文本
                    bar_title="柱状图",
                    # 堆积标题文本
                    stack_title="堆叠",
                    # 平铺标题文
                    tiled_title="平铺",
                ),

                # 工具箱选框组件配置项
                opts.ToolBoxFeatureBrushOpts(  
                    # 选择显示哪些选框
                    type_=[]
                ),
            )
        ),
    )
    return bars


def line():
    '''
    展示每一天收集到的接入车辆总数
    展示每一天每辆车的平均累计里程
    '''
    # 接入车辆总数走势
    sql1 = '''
    select
        date_format(sendtime, "%Y-%m-%d") as time,
        count(distinct vin) as num
    from
        activity a
    group by
        date_format(sendtime, "%Y-%m-%d")
    order by
        time;
    '''
    cur.execute(sql1)
    result1 = cur.fetchall()
    result1 = pd.DataFrame(list(result1),columns=['time','接入数'])

    # 车辆行驶里程
    sql2 = '''
    select
        date_format(sendtime, "%Y-%m-%d") as time,
        (sum(accu_mile) / count(distinct vin)) as mile
    from 
        activity a
    group by
        date_format(sendtime, "%Y-%m-%d")
    order by 
        time;
    '''
    cur.execute(sql2)
    result2 = cur.fetchall()
    result2 = pd.DataFrame(list(result2),columns=['time','累计里程'])

    # 加速踏板行程值
    sql3 = '''
    select
        date_format(sendtime, "%Y-%m-%d") as time,
        (sum(accel_pedal_travel) / count(distinct vin)) as '加速踏板行程均值'
    from 
        activity a
    group by
        date_format(sendtime, "%Y-%m-%d")
    order by 
        time;	
    '''
    cur.execute(sql3)
    result3 = cur.fetchall()
    result3 = pd.DataFrame(list(result3),columns=['time','加速踏板行程均值'])

    sql4 = '''
    select
        date_format(sendtime, "%Y-%m-%d") as time,
        avg(max_bat_mer_V) as '电压最高均值',
        avg(min_bat_mer_V) as '电压最低均值'
    from 
        activity a
    group by
        date_format(sendtime, "%Y-%m-%d")
    order by 
        time;		
    '''
    cur.execute(sql4)
    result4 = cur.fetchall()
    result4 = pd.DataFrame(list(result4),columns=['time','电压最高均值','电压最低均值'])
    result4['电压最低均值'] = result4['电压最低均值'] - 1000 

    # 合并四张表
    result = pd.merge(result1,result2,on='time')
    result = pd.merge(result,result3,on='time')
    result = pd.merge(result,result4,on='time')

    # 折线图创建
    lines = Line(init_opts=opts.InitOpts(theme='dark',chart_id=4)) 

    # 添加值
    lines.add_xaxis(list(result['time'])),
    lines.add_yaxis("接入数", list(result['接入数'])),
    lines.add_yaxis("累计里程", list(result['累计里程'])),
    lines.add_yaxis("加速踏板行程均值", list(result['加速踏板行程均值'])),
    lines.add_yaxis("电压最高均值", list(result['电压最高均值'])),
    lines.add_yaxis("电压最低均值", list(result['电压最低均值'])),

    # 配置
    lines.set_global_opts(

        # 图表标题
        title_opts=opts.TitleOpts(
            # 标题
            title="接入数/累计里程/踏板行程/电压",
            # 标题位置
            pos_left="left",
            # 标题字体大小设置为 20
            title_textstyle_opts=opts.TextStyleOpts(font_size=15)
        ),

        # 图例设置
        legend_opts=opts.LegendOpts(
            # 显示图例，就是各个指标的切换按钮
            is_show=True, 
            # 图例设置为单选模式(multiple 多选)
            selected_mode='single',
            # 图例的位置
            pos_top='5%', pos_right='20%',
            # 图例的布局朝向，水平(vertical 垂直)
            orient='horizontal', 
            # 图例形状
            legend_icon='circle'
        ),

        # 工具箱设置
        toolbox_opts=opts.ToolboxOpts(
            # 是否显示工具栏组件
            is_show=True,
            # 布局朝向('vertical')
            orient='vertical',
            # 离上边距的距离
            pos_top='10%',
            # 离左边距的距离
            pos_left='91%',

            # 配置各个工具箱
            feature=opts.ToolBoxFeatureOpts(

                # 保存工具
                opts.ToolBoxFeatureSaveAsImageOpts(
                    # 是否显示
                    is_show=True,
                    # 提示语
                    title="保存",                 
                ),

                # 还原工具
                opts.ToolBoxFeatureRestoreOpts(
                    # 是否显示该工具
                    is_show=True,
                    # 提示语
                    title="还原", 
                ),

                # 数据视图工具
                opts.ToolBoxFeatureDataViewOpts(
                    # 是否显示该工具
                    is_show=True,
                    # 提示语
                    title="数据视图",
                    # 是否不可编辑
                    is_read_only=False,
                ),

                # 缩放工具配置项，直角坐标系适用
                opts.ToolBoxFeatureDataZoomOpts(
                    # 是否显示该工具。
                    is_show=True,
                    # 提示语
                    zoom_title="区域缩放",
                    # 提示语
                    back_title="缩放还原",
                ),

                # 图表类型切换，适用于直角坐标系
                opts.ToolBoxFeatureMagicTypeOpts(
                    # 是否显示该工具
                    is_show=True,
                    # 启用的动态类型('stack','line','bar','tiled')
                    type_=['stack','line','bar','tiled'],
                    # 折线标题文本
                    line_title="折线图",
                    # 柱状标题文本
                    bar_title="柱状图",
                    # 堆积标题文本
                    stack_title="堆叠",
                    # 平铺标题文
                    tiled_title="平铺",
                ),

                # 工具箱选框组件配置项
                opts.ToolBoxFeatureBrushOpts(  
                    # 选择显示哪些选框
                    type_=[]
                ),
            )
        ),
    )
    return lines
    

def area_line():
    '''
    展示每一天单体电池平均最高，最低温度
    单体电池平均，总电压，总电流，可充电储能装置电压，可充电储能装置电流，绝缘电阻
    电机控制器直流母线电流每天平均电流，控制器平均温度，平均转速，平均转矩，平均温度，平均输入电压
    '''
    sql1 = '''
    select
        date_format(sendtime, "%Y-%m-%d") as time,
        avg(max_TEMP) as '最高温度均值',
        avg(min_TEMP) as '最低温度均值'
    from 
        activity a
    group by
        date_format(sendtime, "%Y-%m-%d")
    order by 
        time;			
    '''
    cur.execute(sql1)
    result1 = cur.fetchall()
    result1 = pd.DataFrame(list(result1),columns=['time','最高温度均值','最低温度均值'])
    result1['最低温度均值'] = result1['最低温度均值'] - 10

    sql2 = '''
    select
        date_format(sendtime, "%Y-%m-%d") as time,
        avg(total_voltage) as '总电压均值',
        avg(total_current) as '总电流均值',
        avg(voltage) as '可充电储能装置电压均值',
        avg(current) as '可充电储能装置电流均值',
        avg(IR) as '绝缘电阻均值'
    from 
        activity a
    group by
        date_format(sendtime, "%Y-%m-%d")
    order by 
        time;					
    '''
    cur.execute(sql2)
    result2 = cur.fetchall()
    result2 = pd.DataFrame(list(result2),columns=['time','总电压均值','总电流均值','可充电储能装置电压均值','可充电储能装置电流均值','绝缘电阻均值'])

    sql3 = '''
    select 
        date_format(sendtime, "%Y-%m-%d") as time,
        avg(motor_CON_DC_bus_C) '平均电流',
        avg(DM_CON_TEMP) '控制器平均温度',
        avg(DM_speed) '平均转速',
        avg(DM_torque) '平均转矩',
        avg(DM_TEMP) '均温度',
        avg(motor_CON_in_V) '平均输入电压'
    from 
        activity a 
    group by
        date_format(sendtime, "%Y-%m-%d")
    order by 
        time ;					
    '''
    cur.execute(sql3)
    result3 = cur.fetchall()
    result3 = pd.DataFrame(list(result3),columns=['time','平均电流','控制器平均温度','平均转速','平均转矩','均温度','平均输入电压'])

    # 合并表
    result = pd.merge(result1,result2,on='time')
    result = pd.merge(result,result3,on='time')

    # 折线图创建
    lines = Line(init_opts=opts.InitOpts(theme='dark',chart_id=5)) 

    # 添加值
    lines.add_xaxis(list(result['time'])),
    lines.add_yaxis("最高温度均值", 
                    list(result['最高温度均值']),
                    # 面积堆叠
                    areastyle_opts=opts.AreaStyleOpts(
                        opacity=0.3
                    ),
                   ),
    lines.add_yaxis("最低温度均值", 
                    list(result['最低温度均值']),
                    # 面积堆叠
                    areastyle_opts=opts.AreaStyleOpts(
                        opacity=0.3
                    ),
                   ),
    lines.add_yaxis("总电压均值", 
                    list(result['总电压均值']),
                    # 面积堆叠
                    areastyle_opts=opts.AreaStyleOpts(
                        opacity=0.3
                    ),
                   ),
    lines.add_yaxis("总电流均值", 
                    list(result['总电流均值']),
                    # 面积堆叠
                    areastyle_opts=opts.AreaStyleOpts(
                        opacity=0.3
                    ),
                   ),
    lines.add_yaxis("平均输入电压", 
                    list(result['平均输入电压']),
                    # 面积堆叠
                    areastyle_opts=opts.AreaStyleOpts(
                        opacity=0.3
                    ),
                   ),
    lines.add_yaxis("可充电储能装置电压均值", 
                    list(result['可充电储能装置电压均值']),
                    # 面积堆叠
                    areastyle_opts=opts.AreaStyleOpts(
                        opacity=0.3
                    ),
                   ),
    lines.add_yaxis("可充电储能装置电流均值", 
                    list(result['可充电储能装置电流均值']),
                    # 面积堆叠
                    areastyle_opts=opts.AreaStyleOpts(
                        opacity=0.3
                    ),
                   ),
    lines.add_yaxis("绝缘电阻均值", 
                    list(result['绝缘电阻均值']),
                    # 面积堆叠
                    areastyle_opts=opts.AreaStyleOpts(
                        opacity=0.3
                    ),
                   ),
    lines.add_yaxis("平均电流", 
                    list(result['平均电流']),
                    # 面积堆叠
                    areastyle_opts=opts.AreaStyleOpts(
                        opacity=0.3
                    ),
                   ),
    lines.add_yaxis("控制器平均温度", 
                    list(result['控制器平均温度']),
                    # 面积堆叠
                    areastyle_opts=opts.AreaStyleOpts(
                        opacity=0.3
                    ),
                   ),
    lines.add_yaxis("平均转速", 
                    list(result['平均转速']),
                    # 面积堆叠
                    areastyle_opts=opts.AreaStyleOpts(
                        opacity=0.3
                    ),
                   ),
    lines.add_yaxis("平均转矩", 
                    list(result['平均转矩']),
                    # 面积堆叠
                    areastyle_opts=opts.AreaStyleOpts(
                        opacity=0.3
                    ),
                   ),
    lines.add_yaxis("平均温度", 
                    list(result['均温度']),
                    # 面积堆叠
                    areastyle_opts=opts.AreaStyleOpts(
                        opacity=0.3
                    ),
                   ),

    # 系列配置
    lines.set_series_opts(

        # 标签
        label_opts=opts.LabelOpts(
            # 是否显示标签
            is_show=False
        ),
    )

    # 配置
    lines.set_global_opts(

        # 是否帖 y 轴
        xaxis_opts=opts.AxisOpts(
            type_="category", 
            boundary_gap=False
        ),

        # 图表标题
        title_opts=opts.TitleOpts(
            # 标题
            title=" 电池单体",
            # 标题位置
            pos_left="left",
            # 标题字体大小设置为 20
            title_textstyle_opts=opts.TextStyleOpts(font_size=20)
        ),

        # 图例设置
        legend_opts=opts.LegendOpts(
            # 显示图例，就是各个指标的切换按钮
            is_show=True, 
            # 图例设置为单选模式(multiple 多选)
            selected_mode='multiple',
            # 图例的位置
            pos_top='5%', pos_right='10%',
            # 图例的布局朝向，水平(vertical 垂直)
            orient='horizontal', 
            # 图例形状
            legend_icon='circle'
        ),

        # 工具箱设置
        toolbox_opts=opts.ToolboxOpts(
            # 是否显示工具栏组件
            is_show=True,
            # 布局朝向('vertical')
            orient='vertical',
            # 离上边距的距离
            pos_top='10%',
            # 离左边距的距离
            pos_left='91%',

            # 配置各个工具箱
            feature=opts.ToolBoxFeatureOpts(

                # 保存工具
                opts.ToolBoxFeatureSaveAsImageOpts(
                    # 是否显示
                    is_show=True,
                    # 提示语
                    title="保存",                 
                ),

                # 还原工具
                opts.ToolBoxFeatureRestoreOpts(
                    # 是否显示该工具
                    is_show=True,
                    # 提示语
                    title="还原", 
                ),

                # 数据视图工具
                opts.ToolBoxFeatureDataViewOpts(
                    # 是否显示该工具
                    is_show=True,
                    # 提示语
                    title="数据视图",
                    # 是否不可编辑
                    is_read_only=False,
                ),

                # 缩放工具配置项，直角坐标系适用
                opts.ToolBoxFeatureDataZoomOpts(
                    # 是否显示该工具。
                    is_show=True,
                    # 提示语
                    zoom_title="区域缩放",
                    # 提示语
                    back_title="缩放还原",
                ),

                # 图表类型切换，适用于直角坐标系
                opts.ToolBoxFeatureMagicTypeOpts(
                    # 是否显示该工具
                    is_show=True,
                    # 启用的动态类型('stack','line','bar','tiled')
                    type_=['stack','line','bar','tiled'],
                    # 折线标题文本
                    line_title="折线图",
                    # 柱状标题文本
                    bar_title="柱状图",
                    # 堆积标题文本
                    stack_title="堆叠",
                    # 平铺标题文
                    tiled_title="平铺",
                ),

                # 工具箱选框组件配置项
                opts.ToolBoxFeatureBrushOpts(  
                    # 选择显示哪些选框
                    type_=[]
                ),
            )
        ),
    )
    return lines
    

def solar1():
    '''
    展示每一天车系占比分布
    '''
    sql = '''
    select
        date_format(sendtime, "%Y-%m-%d") as time,
        car_model ,
        count(distinct vin) as num
    from
        activity a
    group by
        date_format(sendtime, "%Y-%m-%d"),
        car_model
    order by
        time;
    '''
    cur.execute(sql)
    result = cur.fetchall()
    result = pd.DataFrame(list(result),columns=['time','car_model','num'])

    c = (
        Polar(init_opts=opts.InitOpts(theme='dark',chart_id=6))

        # 极坐标轴配置
        .add_schema(

            # 极坐标系角度轴配置项
            angleaxis_opts=opts.AngleAxisOpts(
                # 设置坐标轴
                data=list(result['time'].unique()),
                # 坐标轴方向，顺时针
                is_clockwise=True,            
            )

        )

        # 添加数据
        .add("东风品牌EJ04系列", 
             list(result[result['car_model'] == '东风品牌EJ04系列']['num']), 
             type_="bar", 
             stack="stack0")
        .add("东风品牌F15系列", 
             list(result[result['car_model'] == '东风品牌F15系列']['num']), 
             type_="bar", 
             stack="stack0")
        .add("东风品牌X37PHEV系列", 
             list(result[result['car_model'] == '东风品牌X37PHEV系列']['num']), 
             type_="bar", 
             stack="stack0")

        # 全局配置
        .set_global_opts(

            # 图表标题
            title_opts=opts.TitleOpts(
                # 标题
                title="车系分布",
                # 标题位置
                pos_left="left",
                # 标题字体大小设置为 20
                title_textstyle_opts=opts.TextStyleOpts(font_size=20)
            ),

            # 图例设置
            legend_opts=opts.LegendOpts(
                # 显示图例，就是各个时间的切换按钮
                is_show=True, 
                # 图例设置为多选模式(single 单选)
                selected_mode='multiple',
                # 图例的位置
                pos_top='5%', pos_right='5%',
                # 图例的布局朝向，垂直('horizontal' 水平)
                orient='vertical', 
                # 图例形状
                legend_icon='circle'
            ),

            # 工具箱设置
            toolbox_opts=opts.ToolboxOpts(
                # 是否显示工具栏组件
                is_show=True,
                # 布局朝向，水平(vertical 垂直)
                orient='vertical',
                # 离上边距的距离
                pos_top='30%',
                pos_left='86%',

                # 配置各个工具箱
                feature=opts.ToolBoxFeatureOpts(

                    # 保存工具
                    opts.ToolBoxFeatureSaveAsImageOpts(
                        # 是否显示
                        is_show=True,
                        # 提示语
                        title="保存为图片",                 
                    ),

                    # 还原工具
                    opts.ToolBoxFeatureRestoreOpts(
                        # 是否显示该工具
                        is_show=True,
                        # 提示语
                        title="还原", 
                    ),

                    # 数据视图工具
                    opts.ToolBoxFeatureDataViewOpts(
                        # 是否显示该工具
                        is_show=True,
                        # 提示语
                        title="数据视图",
                        # 是否不可编辑
                        is_read_only=False,
                    ),

                    # 缩放工具配置项，直角坐标系适用
                    opts.ToolBoxFeatureDataZoomOpts(
                        # 是否显示该工具。
                        is_show=False,
                        # 提示语
                        zoom_title="区域缩放",
                        # 提示语
                        back_title="区域缩放还原",
                    ),

                    # 图表类型切换，适用于直角坐标系
                    opts.ToolBoxFeatureMagicTypeOpts(
                        # 是否显示该工具
                        is_show=False,
                        # 启用的动态类型
                        type_=['stack','line','bar','tiled'],                   
                    ),

                    # 工具箱选框组件配置项
                    opts.ToolBoxFeatureBrushOpts(
                        # 选择显示哪些选框
                        type_=[]
                    ),
                )
            ),
        )
    )
    return c 


def solar2():
    '''
    展示电压电池单体代号每一天代号占比，代号以区间划分
    '''
    sql = '''
    select
        date_format(sendtime, "%Y-%m-%d") as time,
        max_V_bat_mer_code as '电压电池单体代号',
        count(max_V_bat_mer_code) as num
    from 
        activity a
    group by
        date_format(sendtime, "%Y-%m-%d"),
        max_V_bat_mer_code
    order by 
        time;	
    '''
    cur.execute(sql)
    result = cur.fetchall()
    result = pd.DataFrame(list(result),columns=['time','电压电池单体代号','num'])

    # 代号分区
    result['电压电池单体代号'] = pd.cut(result['电压电池单体代号'],
                                bins=[0,20,40,60,80,100])
    result['电压电池单体代号'] = result['电压电池单体代号'].astype(str)

    # 分组计算每个区间代号的总和
    result = result.groupby(['time','电压电池单体代号']).sum().reset_index()

    c = (
        Polar(init_opts=opts.InitOpts(theme='dark',chart_id=7))

        # 极坐标轴配置
        .add_schema(

            # 极坐标系角度轴配置项
            angleaxis_opts=opts.AngleAxisOpts(
                # 设置坐标轴
                data=list(result['time'].unique()),
                # 坐标轴方向，顺时针
                is_clockwise=True,            
            )

        )

        # 添加数据
        .add("(0, 20]", 
             list(result[result['电压电池单体代号'] == '(0, 20]']['num']), 
             type_="bar", 
             stack="stack0")
        .add("(20, 40]", 
             list(result[result['电压电池单体代号'] == '(20, 40]']['num']), 
             type_="bar", 
             stack="stack0")
        .add("(40, 60]", 
             list(result[result['电压电池单体代号'] == '(40, 60]']['num']), 
             type_="bar", 
             stack="stack0")
        .add("(60, 80]", 
             list(result[result['电压电池单体代号'] == '(60, 80]']['num']), 
             type_="bar", 
             stack="stack0")
        .add("(80, 100]", 
             list(result[result['电压电池单体代号'] == '(80, 100]']['num']), 
             type_="bar", 
             stack="stack0")

        # 全局配置
        .set_global_opts(

            # 图表标题
            title_opts=opts.TitleOpts(
                # 标题
                title="电池单体代号分布",
                # 标题位置
                pos_left="left",
                # 标题字体大小设置为 20
                title_textstyle_opts=opts.TextStyleOpts(font_size=20)
            ),

            # 图例设置
            legend_opts=opts.LegendOpts(
                # 显示图例，就是各个时间的切换按钮
                is_show=True, 
                # 图例设置为多选模式(single 单选)
                selected_mode='multiple',
                # 图例的位置
                pos_top='5%', pos_right='5%',
                # 图例的布局朝向，垂直('horizontal' 水平)
                orient='vertical', 
                # 图例形状
                legend_icon='circle'
            ),

            # 工具箱设置
            toolbox_opts=opts.ToolboxOpts(
                # 是否显示工具栏组件
                is_show=True,
                # 布局朝向，水平(vertical 垂直)
                orient='vertical',
                # 离上边距的距离
                pos_top='30%',
                pos_left='86%',

                # 配置各个工具箱
                feature=opts.ToolBoxFeatureOpts(

                    # 保存工具
                    opts.ToolBoxFeatureSaveAsImageOpts(
                        # 是否显示
                        is_show=True,
                        # 提示语
                        title="保存为图片",                 
                    ),

                    # 还原工具
                    opts.ToolBoxFeatureRestoreOpts(
                        # 是否显示该工具
                        is_show=True,
                        # 提示语
                        title="还原", 
                    ),

                    # 数据视图工具
                    opts.ToolBoxFeatureDataViewOpts(
                        # 是否显示该工具
                        is_show=True,
                        # 提示语
                        title="数据视图",
                        # 是否不可编辑
                        is_read_only=False,
                    ),

                    # 缩放工具配置项，直角坐标系适用
                    opts.ToolBoxFeatureDataZoomOpts(
                        # 是否显示该工具。
                        is_show=False,
                        # 提示语
                        zoom_title="区域缩放",
                        # 提示语
                        back_title="区域缩放还原",
                    ),

                    # 图表类型切换，适用于直角坐标系
                    opts.ToolBoxFeatureMagicTypeOpts(
                        # 是否显示该工具
                        is_show=False,
                        # 启用的动态类型
                        type_=['stack','line','bar','tiled'],                   
                    ),

                    # 工具箱选框组件配置项
                    opts.ToolBoxFeatureBrushOpts(
                        # 选择显示哪些选框
                        type_=[]
                    ),
                )
            ),
        )
    )
    return c 


def solar3():
    '''
    温度探针序号每一天分布，以区间划分序号
    '''
    sql = '''
    select
        date_format(sendtime, "%Y-%m-%d") as time,
        max_TEMP_probe_serial as '温度探针序号',
        count(max_TEMP_probe_serial) as num
    from 
        activity a
    group by
        date_format(sendtime, "%Y-%m-%d"),
        max_TEMP_probe_serial
    order by 
        time;	
    '''
    cur.execute(sql)
    result = cur.fetchall()
    result = pd.DataFrame(list(result),columns=['time','温度探针序号','num'])

    # 代号分区
    result['温度探针序号'] = pd.cut(result['温度探针序号'],
                                bins=[0,20,40,60])
    result['温度探针序号'] = result['温度探针序号'].astype(str)

    # 分组计算每个区间代号的总和
    result = result.groupby(['time','温度探针序号']).sum().reset_index()

    c = (
        Polar(init_opts=opts.InitOpts(theme='dark',chart_id=8))

        # 极坐标轴配置
        .add_schema(

            # 极坐标系角度轴配置项
            angleaxis_opts=opts.AngleAxisOpts(
                # 设置坐标轴
                data=list(result['time'].unique()),
                # 坐标轴方向，顺时针
                is_clockwise=True,            
            )

        )

        # 添加数据
        .add("(0, 20]", 
             list(result[result['温度探针序号'] == '(0, 20]']['num']), 
             type_="bar", 
             stack="stack0")
        .add("(20, 40]", 
             list(result[result['温度探针序号'] == '(20, 40]']['num']), 
             type_="bar", 
             stack="stack0")
        .add("(40, 60]", 
             list(result[result['温度探针序号'] == '(40, 60]']['num']), 
             type_="bar", 
             stack="stack0")

        # 全局配置
        .set_global_opts(

            # 图表标题
            title_opts=opts.TitleOpts(
                # 标题
                title="温度探针序号分布",
                # 标题位置
                pos_left="left",
                # 标题字体大小设置为 20
                title_textstyle_opts=opts.TextStyleOpts(font_size=20)
            ),

            # 图例设置
            legend_opts=opts.LegendOpts(
                # 显示图例，就是各个时间的切换按钮
                is_show=True, 
                # 图例设置为多选模式(single 单选)
                selected_mode='multiple',
                # 图例的位置
                pos_top='5%', pos_right='5%',
                # 图例的布局朝向，垂直('horizontal' 水平)
                orient='vertical', 
                # 图例形状
                legend_icon='circle'
            ),

            # 工具箱设置
            toolbox_opts=opts.ToolboxOpts(
                # 是否显示工具栏组件
                is_show=True,
                # 布局朝向，水平(vertical 垂直)
                orient='vertical',
                # 离上边距的距离
                pos_top='30%',
                pos_left='86%',

                # 配置各个工具箱
                feature=opts.ToolBoxFeatureOpts(

                    # 保存工具
                    opts.ToolBoxFeatureSaveAsImageOpts(
                        # 是否显示
                        is_show=True,
                        # 提示语
                        title="保存为图片",                 
                    ),

                    # 还原工具
                    opts.ToolBoxFeatureRestoreOpts(
                        # 是否显示该工具
                        is_show=True,
                        # 提示语
                        title="还原", 
                    ),

                    # 数据视图工具
                    opts.ToolBoxFeatureDataViewOpts(
                        # 是否显示该工具
                        is_show=True,
                        # 提示语
                        title="数据视图",
                        # 是否不可编辑
                        is_read_only=False,
                    ),

                    # 缩放工具配置项，直角坐标系适用
                    opts.ToolBoxFeatureDataZoomOpts(
                        # 是否显示该工具。
                        is_show=False,
                        # 提示语
                        zoom_title="区域缩放",
                        # 提示语
                        back_title="区域缩放还原",
                    ),

                    # 图表类型切换，适用于直角坐标系
                    opts.ToolBoxFeatureMagicTypeOpts(
                        # 是否显示该工具
                        is_show=False,
                        # 启用的动态类型
                        type_=['stack','line','bar','tiled'],                   
                    ),

                    # 工具箱选框组件配置项
                    opts.ToolBoxFeatureBrushOpts(
                        # 选择显示哪些选框
                        type_=[]
                    ),
                )
            ),
        )
    )
    return c 
    

def solar4():
    '''
    每一天可用单体电池总数分布占比
    '''
    sql = '''
    select
        date_format(sendtime, "%Y-%m-%d") as time,
        total_mer_battery,
        count(total_mer_battery) as num
    from 
        activity a
    group by
        date_format(sendtime, "%Y-%m-%d"),
        total_mer_battery
    order by 
        time;		
    '''
    cur.execute(sql)
    result = cur.fetchall()
    result = pd.DataFrame(list(result),columns=['time','total_mer_battery','num'])

    c = (
        Polar(init_opts=opts.InitOpts(theme='dark',chart_id=9))

        # 极坐标轴配置
        .add_schema(

            # 极坐标系角度轴配置项
            angleaxis_opts=opts.AngleAxisOpts(
                # 设置坐标轴
                data=list(result['time'].unique()),
                # 坐标轴方向，顺时针
                is_clockwise=True,            
            )

        )

        # 添加数据
        .add("84", 
             list(result[result['total_mer_battery'] == 84]['num']), 
             type_="bar", 
             stack="stack0")
        .add("94", 
             list(result[result['total_mer_battery'] == 94]['num']), 
             type_="bar", 
             stack="stack0")
        .add("96", 
             list(result[result['total_mer_battery'] == 96]['num']), 
             type_="bar", 
             stack="stack0")

        # 全局配置
        .set_global_opts(

            # 图表标题
            title_opts=opts.TitleOpts(
                # 标题
                title="温度探针序号分布",
                # 标题位置
                pos_left="left",
                # 标题字体大小设置为 20
                title_textstyle_opts=opts.TextStyleOpts(font_size=20)
            ),

            # 图例设置
            legend_opts=opts.LegendOpts(
                # 显示图例，就是各个时间的切换按钮
                is_show=True, 
                # 图例设置为多选模式(single 单选)
                selected_mode='multiple',
                # 图例的位置
                pos_top='5%', pos_right='5%',
                # 图例的布局朝向，垂直('horizontal' 水平)
                orient='vertical', 
                # 图例形状
                legend_icon='circle'
            ),

            # 工具箱设置
            toolbox_opts=opts.ToolboxOpts(
                # 是否显示工具栏组件
                is_show=True,
                # 布局朝向，水平(vertical 垂直)
                orient='vertical',
                # 离上边距的距离
                pos_top='30%',
                pos_left='86%',

                # 配置各个工具箱
                feature=opts.ToolBoxFeatureOpts(

                    # 保存工具
                    opts.ToolBoxFeatureSaveAsImageOpts(
                        # 是否显示
                        is_show=True,
                        # 提示语
                        title="保存为图片",                 
                    ),

                    # 还原工具
                    opts.ToolBoxFeatureRestoreOpts(
                        # 是否显示该工具
                        is_show=True,
                        # 提示语
                        title="还原", 
                    ),

                    # 数据视图工具
                    opts.ToolBoxFeatureDataViewOpts(
                        # 是否显示该工具
                        is_show=True,
                        # 提示语
                        title="数据视图",
                        # 是否不可编辑
                        is_read_only=False,
                    ),

                    # 缩放工具配置项，直角坐标系适用
                    opts.ToolBoxFeatureDataZoomOpts(
                        # 是否显示该工具。
                        is_show=False,
                        # 提示语
                        zoom_title="区域缩放",
                        # 提示语
                        back_title="区域缩放还原",
                    ),

                    # 图表类型切换，适用于直角坐标系
                    opts.ToolBoxFeatureMagicTypeOpts(
                        # 是否显示该工具
                        is_show=False,
                        # 启用的动态类型
                        type_=['stack','line','bar','tiled'],                   
                    ),

                    # 工具箱选框组件配置项
                    opts.ToolBoxFeatureBrushOpts(
                        # 选择显示哪些选框
                        type_=[]
                    ),
                )
            ),
        )
    )
    return c 
   

def reversed_bar():
    '''
    展示每一天各报警的车辆数
    '''
    sql = '''
    select 
        date_format(sendtime, "%Y-%m-%d") as time,
        count(distinct vin,case when car_modeltype = 'DFM7000G1F3BEV' then car_modeltype end) as '类型1',
        count(distinct vin,case when car_modeltype = 'DFM7000G1F6BEV' then car_modeltype end) as '类型2',
        count(distinct vin,case when car_modeltype = 'DFM7000G1F2BEV' then car_modeltype end) as '类型3',
        count(distinct vin,case when car_modeltype = 'DFM7000H2DBEV' then car_modeltype end) as '类型4',
        count(distinct vin,case when car_modeltype = 'DFM7000H2ABEV1' then car_modeltype end) as '类型5',
        count(distinct vin,case when car_modeltype = 'DFM7000H2DBEV1' then car_modeltype end) as '类型6',
        count(distinct vin,case when car_modeltype = 'DFM7000G1F7BEV' then car_modeltype end) as '类型7',
        count(distinct vin,case when car_modeltype = 'DFM7000G1ABEV' then car_modeltype end) as '类型8',
        count(distinct vin,case when car_modeltype = 'DFM7000H2ABEV' then car_modeltype end) as '类型9',
        count(distinct vin,case when car_modeltype = 'DFM7000G1BBEV' then car_modeltype end) as '类型10',
        count(distinct vin,case when car_modeltype = 'DFM6460D5F1CHEV' then car_modeltype end) as '类型11',
        count(distinct vin,case when car_modeltype = 'DFM7000G1ABEV1' then car_modeltype end) as '类型12'
    from 
        activity a 
    group by
        date_format(sendtime, "%Y-%m-%d")
    order by 
        time ;		
    '''
    cur.execute(sql)
    result = cur.fetchall()
    result = pd.DataFrame(list(result),columns=['time','类型1','类型2','类型3','类型4','类型5','类型6','类型7','类型8','类型9','类型10','类型11','类型12'])

    # 柱状图创建
    bars = Bar(init_opts=opts.InitOpts(theme='dark',chart_id=10)) 

    # 添加值
    bars.add_xaxis(list(result['time'])),
    bars.add_yaxis("类型1", list(result['类型1'])),
    bars.add_yaxis("类型2", list(result['类型2'])),
    bars.add_yaxis("类型3", list(result['类型3'])),
    bars.add_yaxis("类型4", list(result['类型4'])),
    bars.add_yaxis("类型5", list(result['类型5'])),
    bars.add_yaxis("类型6", list(result['类型6'])),
    bars.add_yaxis("类型7", list(result['类型7'])),
    bars.add_yaxis("类型8", list(result['类型8'])),
    bars.add_yaxis("类型9", list(result['类型9'])),
    bars.add_yaxis("类型10", list(result['类型10'])),
    bars.add_yaxis("类型11", list(result['类型11'])),
    bars.add_yaxis("类型12", list(result['类型12'])),

    # 翻转 xy 轴
    bars.reversal_axis()

    # 系列配置
    bars.set_series_opts(

        # 标签配置
        label_opts=opts.LabelOpts(
            # 标签位置
            position="right"
        )
    )

    # 全局配置
    bars.set_global_opts(

        # 图表标题
        title_opts=opts.TitleOpts(
            # 标题
            title="车辆出行类型 ",
            # 标题位置
            pos_left="left",
            # 标题字体大小设置为 20
            title_textstyle_opts=opts.TextStyleOpts(font_size=20)
        ),

        # 图例设置
        legend_opts=opts.LegendOpts(
            # 显示图例，就是各个指标的切换按钮
            is_show=True, 
            # 图例设置为单选模式(multiple 多选)
            selected_mode='multiple',
            # 图例的位置
            pos_top='5%',# pos_right='50%',
            # 图例的布局朝向，水平(vertical 垂直)
            orient='horizontal', 
            # 图例形状
            legend_icon='circle'
        ),

        # 工具箱设置
        toolbox_opts=opts.ToolboxOpts(
            # 是否显示工具栏组件
            is_show=True,
            # 布局朝向('vertical')
            orient='vertical',
            # 离上边距的距离
            pos_top='10%',
            # 离左边距的距离
            pos_left='91%',

            # 配置各个工具箱
            feature=opts.ToolBoxFeatureOpts(

                # 保存工具
                opts.ToolBoxFeatureSaveAsImageOpts(
                    # 是否显示
                    is_show=True,
                    # 提示语
                    title="保存",                 
                ),

                # 还原工具
                opts.ToolBoxFeatureRestoreOpts(
                    # 是否显示该工具
                    is_show=True,
                    # 提示语
                    title="还原", 
                ),

                # 数据视图工具
                opts.ToolBoxFeatureDataViewOpts(
                    # 是否显示该工具
                    is_show=True,
                    # 提示语
                    title="数据视图",
                    # 是否不可编辑
                    is_read_only=False,
                ),

                # 缩放工具配置项，直角坐标系适用
                opts.ToolBoxFeatureDataZoomOpts(
                    # 是否显示该工具。
                    is_show=True,
                    # 提示语
                    zoom_title="区域缩放",
                    # 提示语
                    back_title="缩放还原",
                ),

                # 图表类型切换，适用于直角坐标系
                opts.ToolBoxFeatureMagicTypeOpts(
                    # 是否显示该工具
                    is_show=True,
                    # 启用的动态类型('stack','line','bar','tiled')
                    type_=['stack','line','bar','tiled'],
                    # 折线标题文本
                    line_title="折线图",
                    # 柱状标题文本
                    bar_title="柱状图",
                    # 堆积标题文本
                    stack_title="堆叠",
                    # 平铺标题文
                    tiled_title="平铺",
                ),

                # 工具箱选框组件配置项
                opts.ToolBoxFeatureBrushOpts(  
                    # 选择显示哪些选框
                    type_=[]
                ),
            )
        ),
    )
    return bars
    
    
def page_draggable_layout():
    page = Page(layout=Page.DraggablePageLayout)
    page.add(
        heat_map(),
        bar(),
        stack_bar(),
        line(),
        area_line(),
        solar1(),
        solar2(),
        #solar3(),
        #solar4(),
        reversed_bar()
    )
    page.render("../output/page_draggable_layout.html")


if __name__ == "__main__":
    page_draggable_layout()
    Page.save_resize_html('../output/page_draggable_layout.html',
                          cfg_file='../output/chart_config.json',
                         dest='../output/dashbord.html')

D:\programming\dev_env\anaconda\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
D:\programming\dev_env\anaconda\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
D:\programming\dev_env\anaconda\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
D:\programming\dev_env\anaconda\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
D:\programming\dev_env\anaconda\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强